# Data from NOAA and climod2
* https://www.ncdc.noaa.gov/cdo-web/
* http://climod2.nrcc.cornell.edu/
* Note:
    * T: Trace amount- Less than 0.01" precipitation; less than 0.1" snowfall; less than 1" snow depth.
    * S: Subsequent- Indicates the observation is missing, but is included in a subsequent value.
    * A: Accumulated- Indicates an accumulated value which includes the current day and any immediately preceeding missing days (starting with a day flagged by "S").
    * M: Missing.

___

# Importing Libraries and data

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import os
sns.set()

#from fbprophet import Prophet

In [2]:
df = pd.read_csv('Bradley_1_8_1942_2_8_2023.csv', skipinitialspace = True)

In [3]:
df_cornell = pd.read_csv('Bradley_Cornell_1_1_1940_2_13_2023.txt', sep=' ', header=None)

In [6]:
df = df.drop(['STATION'], axis = 1)

In [11]:
df = df.drop(['NAME'], axis = 1)

In [13]:
# Group by year to see how many blanks
df['Year'] = pd.DatetimeIndex(df['DATE']).year
df['Month'] = pd.DatetimeIndex(df['DATE']).month
df['Day'] = pd.DatetimeIndex(df['DATE']).day

In [15]:
df_year_blank = df.Year.isnull().groupby([df['ACMH'],df['ACSH'],df['AWND'],df['FMTM'],df['FRGT'],
                              df['GAHT'],df['PGTM'],df['PRCP'],df['PSUN'],df['SNOW'],
                              df['TAVG'],df['TMAX'],df['TMIN'],df['TSUN'],df['WDF1'],
                              df['WDF2'],df['WDF5'],df['WDFG'],df['WDFM'],df['WESD'],
                              df['WSF1'],df['WSF2'],df['WSF5'],df['WSFG'],df['WSFM'],
                              df['WT01'],df['WT02'],df['WT03'],df['WT04'],df['WT05'],
                              df['WT06'],df['WT07'],df['WT08'],df['WT09'],df['WT11'],
                              df['WT12'],df['WT13'],df['WT14'],df['WT15'],df['WT16'],
                              df['WT17'],df['WT18'],df['WT19'],df['WT21'],df['WT22'],
                              df['WV01'],df['WV03']]).sum().astype(int).reset_index(name='count')

Index(['DATE', 'ACMH', 'ACSH', 'AWND', 'FMTM', 'FRGT', 'GAHT', 'PGTM', 'PRCP',
       'PSUN', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN', 'TSUN', 'WDF1', 'WDF2',
       'WDF5', 'WDFG', 'WDFM', 'WESD', 'WSF1', 'WSF2', 'WSF5', 'WSFG', 'WSFM',
       'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09',
       'WT11', 'WT12', 'WT13', 'WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WT19',
       'WT21', 'WT22', 'WV01', 'WV03', 'Year', 'Month', 'Day'],
      dtype='object')

In [26]:
df_year_blank = df.TAVG.isnull().groupby([df['Year']]).sum().astype(int).reset_index(name='count')

In [44]:
# Can't use 1942 to 1945,
df_year_blank.iloc[50:80]

,Year,count
50,1995,365
51,1996,366
52,1997,365
53,1998,92
54,1999,2
55,2000,2
56,2001,0
57,2002,1
58,2003,0
59,2004,0


In [39]:
for i in df.columns:
    print(i, df[i].isnull().sum()/len(df))

DATE 0.0
ACMH 0.5988958824149935
ACSH 0.5987200675129224
AWND 0.4979078026653539
FMTM 0.6431660747564963
FRGT 0.9998593480783431
GAHT 0.9998945110587574
PGTM 0.5575090544674567
PRCP 0.0483842610499666
PSUN 0.7560040789057281
SNOW 0.12792292274693204
SNWD 0.30440592144590173
TAVG 0.7311086887724604
TMAX 0.04901719469742255
TMIN 0.04898203171700834
TSUN 0.5607088856851506
WDF1 0.8460916347269595
WDF2 0.6551566510777453
WDF5 0.655754421744787
WDFG 0.7296670065754773
WDFM 0.7532965294138331
WESD 0.9402229332958262
WSF1 0.846021308766131
WSF2 0.6551566510777453
WSF5 0.655754421744787
WSFG 0.729456028692992
WSFM 0.7532965294138331
WT01 0.6350785892612257
WT02 0.94883786349731
WT03 0.9468687365941137
WT04 0.9746474911213474
WT05 0.997925384155561
WT06 0.9798164492422378
WT07 0.9999648370195858
WT08 0.8146910932170611
WT09 0.9954639755265656
WT11 0.9992615774113014
WT12 0.9999648370195858
WT13 0.9075213615106016
WT14 0.967052287351876
WT15 0.9969759836843771
WT16 0.6768522099933191
WT17 0.9943

### The Precipitation (PRCP) from the NOAA dataset is better because it has no missing or T or M so I will merge that with this dataset

In [3]:
#df2 = pd.read_csv('Bradley_NOAA_1.1.1980_12.1.2020.csv', usecols = ['DATE', 'PRCP'])

In [4]:
#df2 = df2.rename(columns = {'DATE': 'Date'})

In [5]:
#df2

In [6]:
#df1 = pd.merge(df1, df2, on = 'Date', how = 'inner')

## Count how many rows still have Snowfall as T or M

In [7]:
snowfall_t_new = df.loc[df['Snowfall'] == 'T']
print(len(snowfall_t_new))
snowfall_m_new = df.loc[df['Snowfall'] == 'M']
print(len(snowfall_m_new))

0
0


### How many T and M in SnowDepth

In [8]:
snowdepth_t = df.loc[df['SnowDepth'] == 'T']
print(len(snowdepth_t))
snowdepth_m = df.loc[df['SnowDepth'] == 'M']
print(len(snowdepth_m))

0
2188


### How many T and M in Precipitation

In [9]:
precip_t = df.loc[df['Precipitation'] == 'T']
print(len(precip_t))
precip_m = df.loc[df['Precipitation'] == 'M']
print(len(precip_m))

0
0


### How many T and M in PRCP

In [10]:
#precip_t = df1.loc[df1['PRCP'] == 'T']
#print(len(precip_t))
#precip_m = df1.loc[df1['PRCP'] == 'M']
#print(len(precip_m))

### Drop Precipitation and rename PRCP Precipitation

In [11]:
#df1 = df1.drop(['Precipitation'], axis = 1)

In [12]:
#df1 = df1.rename(columns = {'PRCP': 'Precipitation'})

In [13]:
precip_t = df.loc[df['Precipitation'] == 'T']
print(len(precip_t))
precip_m = df.loc[df['Precipitation'] == 'M']
print(len(precip_m))

0
0


In [14]:
#df2 = pd.read_csv('Bradley_12.2.2020_12.27.2020.csv', skipinitialspace = True)

In [15]:
#df = df1.append(df2)

In [16]:
#df

### Add a column for the Year and a column for the Month

In [17]:
df['Year'] = pd.DatetimeIndex(df['Date']).year
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Day'] = pd.DatetimeIndex(df['Date']).day

___

# Exploratory Data Analysis

In [18]:
df.dtypes

Date                        object
MaxTemperature               int64
MaxTemperatureNormal       float64
MaxTemperatureDeparture    float64
MinTemperature               int64
MinTemperatureNormal       float64
MinTemperatureDeparture    float64
AvgTemperature             float64
AvgTemperatureNormal       float64
AvgTemperatureDeparture    float64
AtObsTemperature            object
PrecipitationNormal         object
PrecipitationDeparture      object
Snowfall                   float64
SnowfallNormal              object
SnowfallDeparture           object
SnowDepth                   object
HDD                          int64
HDDNormal                  float64
HDDDeparture               float64
CDD                          int64
CDDNormal                  float64
CDDDeparture               float64
GDD                          int64
Precipitation              float64
Year                         int64
Month                        int64
Day                          int64
dtype: object

### I am going to convert everything except the below to Float64
* AtObsTemperature            object
* Precipitation               object
* PrecipitationNormal         object
* PrecipitationDeparture      object
* Snowfall                    object
* SnowfallNormal              object
* SnowfallDeparture           object
* SnowDepth                   object

In [19]:
#df[['MaxTemperature', 'MinTemperature', 'HDD', 'HDDNormal', 'HDDDeparture', 'CDD', 'CDDNormal', 'CDDDeparture', 'GDD']] = df[['MaxTemperature', 'MinTemperature', 'HDD', 'HDDNormal', 'HDDDeparture', 'CDD', 'CDDNormal', 'CDDDeparture', 'GDD']].apply(pd.to_float)
df[['MaxTemperature', 'MinTemperature', 'HDD', 'HDDNormal', 'HDDDeparture', 'CDD', 'CDDNormal', 'CDDDeparture', 'GDD']] = df[['MaxTemperature', 'MinTemperature', 'HDD', 'HDDNormal', 'HDDDeparture', 'CDD', 'CDDNormal', 'CDDDeparture', 'GDD']].astype(float)

In [20]:
cols = []
for i in df.columns:
    cols.append(i)
print(cols)

['Date', 'MaxTemperature', 'MaxTemperatureNormal', 'MaxTemperatureDeparture', 'MinTemperature', 'MinTemperatureNormal', 'MinTemperatureDeparture', 'AvgTemperature', 'AvgTemperatureNormal', 'AvgTemperatureDeparture', 'AtObsTemperature', 'PrecipitationNormal', 'PrecipitationDeparture', 'Snowfall', 'SnowfallNormal', 'SnowfallDeparture', 'SnowDepth', 'HDD', 'HDDNormal', 'HDDDeparture', 'CDD', 'CDDNormal', 'CDDDeparture', 'GDD', 'Precipitation', 'Year', 'Month', 'Day']


### Null Values
* Result: No missing values

In [22]:
df.isnull().sum()

Date                       0
MaxTemperature             0
MaxTemperatureNormal       0
MaxTemperatureDeparture    0
MinTemperature             0
MinTemperatureNormal       0
MinTemperatureDeparture    0
AvgTemperature             0
AvgTemperatureNormal       0
AvgTemperatureDeparture    0
AtObsTemperature           0
PrecipitationNormal        0
PrecipitationDeparture     0
Snowfall                   0
SnowfallNormal             0
SnowfallDeparture          0
SnowDepth                  0
HDD                        0
HDDNormal                  0
HDDDeparture               0
CDD                        0
CDDNormal                  0
CDDDeparture               0
GDD                        0
Precipitation              0
Year                       0
Month                      0
Day                        0
dtype: int64

### Check to see if the data has S or A
* Result: No S or A

In [23]:
s = df.select_dtypes([np.object]).apply(lambda x: x.str.contains('S').any())
s

<ipython-input-23-260d72f5291a>:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  s = df.select_dtypes([np.object]).apply(lambda x: x.str.contains('S').any())


Date                      False
AtObsTemperature          False
PrecipitationNormal       False
PrecipitationDeparture    False
SnowfallNormal            False
SnowfallDeparture         False
SnowDepth                 False
dtype: bool

In [24]:
a = df.select_dtypes([np.object]).apply(lambda x: x.str.contains('A').any())
a

<ipython-input-24-3d0ccb3cc8d9>:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  a = df.select_dtypes([np.object]).apply(lambda x: x.str.contains('A').any())


Date                      False
AtObsTemperature          False
PrecipitationNormal       False
PrecipitationDeparture    False
SnowfallNormal            False
SnowfallDeparture         False
SnowDepth                 False
dtype: bool

In [25]:
t = df.select_dtypes([np.object]).apply(lambda x: x.str.contains('T').any())
t

<ipython-input-25-da57d6741c05>:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  t = df.select_dtypes([np.object]).apply(lambda x: x.str.contains('T').any())


Date                      False
AtObsTemperature          False
PrecipitationNormal       False
PrecipitationDeparture    False
SnowfallNormal            False
SnowfallDeparture         False
SnowDepth                 False
dtype: bool

In [26]:
m = df.select_dtypes([np.object]).apply(lambda x: x.str.contains('M').any())
m

<ipython-input-26-38d4922a890c>:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  m = df.select_dtypes([np.object]).apply(lambda x: x.str.contains('M').any())


Date                      False
AtObsTemperature           True
PrecipitationNormal        True
PrecipitationDeparture     True
SnowfallNormal             True
SnowfallDeparture          True
SnowDepth                  True
dtype: bool

### How many values are there for MaxTemperature and what are they

In [27]:
print(df['MaxTemperature'].nunique())
snow = df['MaxTemperature'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

100
[  3.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.  16.  17.
  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.  30.  31.
  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.  45.
  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.
  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.  71.  72.  73.
  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.  84.  85.  86.  87.
  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.  98.  99. 100. 101.
 102. 103.]


### How many values are there for MaxTemperatureNormal and what are they

In [28]:
print(df['MaxTemperatureNormal'].nunique())
snow = df['MaxTemperatureNormal'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

333
[34.1 34.2 34.3 34.4 34.5 34.6 34.7 34.8 34.9 35.1 35.2 35.4 35.5 35.6
 35.7 35.8 35.9 36.  36.2 36.4 36.5 36.6 36.7 36.8 36.9 37.1 37.2 37.3
 37.5 37.7 37.8 38.  38.1 38.2 38.4 38.5 38.7 39.  39.2 39.4 39.5 39.7
 39.8 40.  40.1 40.3 40.4 40.6 40.8 40.9 41.2 41.5 41.8 41.9 42.1 42.2
 42.3 42.4 42.7 43.  43.1 43.3 43.5 43.6 43.9 44.  44.3 44.4 44.6 44.8
 45.  45.2 45.3 45.6 45.7 46.  46.4 46.5 46.8 46.9 47.1 47.3 47.5 47.7
 47.9 48.2 48.3 48.6 48.7 49.  49.1 49.4 49.5 49.9 50.3 50.7 51.1 51.5
 51.9 52.  52.3 52.4 52.7 52.8 53.1 53.3 53.5 53.7 53.9 54.2 54.3 54.6
 54.7 55.  55.1 55.4 55.5 55.8 56.  56.2 56.4 56.5 56.9 57.2 57.3 57.6
 57.8 58.  58.2 58.3 58.7 59.  59.1 59.4 59.5 59.8 60.  60.1 60.4 60.5
 60.8 60.9 61.2 61.3 61.5 61.7 61.9 62.1 62.3 62.5 62.6 63.  63.4 63.8
 64.1 64.2 64.5 64.6 64.9 65.  65.3 65.4 65.7 65.8 66.  66.1 66.2 66.4
 66.5 66.6 66.7 66.9 67.  67.1 67.3 67.4 67.7 67.8 68.  68.1 68.3 68.4
 68.5 68.7 68.8 68.9 69.  69.1 69.3 69.4 69.5 69.6 69.7 69.9 70.  70.1
 7

### How many values are there for MaxTemperatureDeparture and what are they

In [29]:
print(df['MaxTemperatureDeparture'].nunique())
snow = df['MaxTemperatureDeparture'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

564
[-32.9 -31.8 -29.2 -28.7 -28.5 -28.2 -28.1 -28.  -27.9 -27.  -26.7 -26.5
 -26.4 -26.2 -26.  -25.7 -25.6 -25.2 -25.1 -25.  -24.5 -24.4 -24.1 -24.
 -23.9 -23.8 -23.7 -23.6 -23.5 -23.4 -23.2 -23.1 -23.  -22.9 -22.8 -22.7
 -22.6 -22.5 -22.4 -22.3 -22.2 -22.  -21.9 -21.8 -21.7 -21.6 -21.5 -21.4
 -21.3 -21.2 -21.1 -21.  -20.9 -20.8 -20.7 -20.4 -20.3 -20.2 -20.1 -20.
 -19.9 -19.8 -19.7 -19.6 -19.5 -19.4 -19.3 -19.2 -19.1 -19.  -18.9 -18.8
 -18.7 -18.6 -18.5 -18.4 -18.3 -18.2 -18.1 -18.  -17.9 -17.8 -17.7 -17.6
 -17.5 -17.4 -17.3 -17.2 -17.1 -17.  -16.9 -16.8 -16.7 -16.6 -16.5 -16.4
 -16.3 -16.2 -16.1 -16.  -15.9 -15.8 -15.7 -15.6 -15.5 -15.4 -15.3 -15.2
 -15.1 -15.  -14.9 -14.8 -14.7 -14.6 -14.5 -14.4 -14.3 -14.2 -14.1 -14.
 -13.9 -13.8 -13.7 -13.6 -13.5 -13.4 -13.3 -13.2 -13.1 -13.  -12.9 -12.8
 -12.7 -12.6 -12.5 -12.4 -12.3 -12.2 -12.1 -12.  -11.9 -11.8 -11.7 -11.6
 -11.5 -11.4 -11.3 -11.2 -11.1 -11.  -10.9 -10.8 -10.7 -10.6 -10.5 -10.4
 -10.3 -10.2 -10.1 -10.   -9.9  -9.8  -9.7  -9.6  

### How many values are there for MinTemperature and what are they

In [30]:
print(df['MinTemperature'].nunique())
snow = df['MinTemperature'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

93
[-21. -14. -13. -12. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.  -2.  -1.
   0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.]


### How many values are there for MinTemperatureNormal and what are they

In [31]:
print(df['MinTemperatureNormal'].nunique())
snow = df['MinTemperatureNormal'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

326
[17.2 17.3 17.4 17.5 17.6 17.7 17.8 17.9 18.  18.1 18.2 18.3 18.4 18.6
 18.7 18.8 18.9 19.  19.2 19.3 19.4 19.5 19.7 19.8 19.9 20.  20.2 20.3
 20.4 20.6 20.7 20.9 21.1 21.2 21.3 21.5 21.7 21.8 21.9 22.1 22.2 22.3
 22.5 22.7 22.9 23.1 23.2 23.3 23.5 23.6 23.7 23.9 24.  24.2 24.3 24.4
 24.6 24.7 24.9 25.  25.2 25.4 25.7 25.8 25.9 26.1 26.2 26.5 26.8 26.9
 27.1 27.2 27.4 27.6 27.7 28.  28.3 28.6 28.7 28.9 29.  29.3 29.4 29.6
 29.7 30.  30.3 30.4 30.7 31.  31.3 31.4 31.6 31.7 31.9 32.1 32.2 32.5
 32.7 32.8 33.  33.2 33.3 33.5 33.6 33.8 34.  34.3 34.5 34.7 35.  35.1
 35.2 35.4 35.5 35.6 35.8 35.9 36.1 36.2 36.3 36.5 36.6 36.7 36.9 37.2
 37.3 37.4 37.6 37.7 37.9 38.  38.1 38.3 38.4 38.6 38.7 38.8 39.1 39.4
 39.6 39.7 39.9 40.1 40.2 40.4 40.5 40.7 40.8 41.  41.1 41.3 41.4 41.6
 41.8 41.9 42.1 42.2 42.4 42.5 42.8 42.9 43.1 43.2 43.3 43.4 43.5 43.6
 43.7 43.9 44.  44.2 44.3 44.5 44.7 44.8 44.9 45.1 45.2 45.4 45.5 45.6
 45.8 45.9 46.1 46.2 46.3 46.4 46.5 46.6 46.7 46.8 47.  47.1 47.3 47.4
 4

### How many values are there for MinTemperatureDeparture and what are they

In [32]:
print(df['MinTemperatureDeparture'].nunique())
snow = df['MinTemperatureDeparture'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

535
[-38.5 -34.  -33.3 -32.9 -32.7 -32.2 -31.4 -31.3 -31.2 -31.1 -30.7 -30.
 -29.6 -29.2 -28.5 -28.  -27.9 -27.6 -27.5 -27.4 -27.1 -26.7 -26.6 -26.4
 -26.3 -26.2 -26.1 -25.9 -25.8 -25.7 -25.6 -25.5 -25.4 -25.3 -25.2 -25.1
 -24.9 -24.6 -24.5 -24.4 -24.3 -24.2 -24.1 -24.  -23.9 -23.7 -23.6 -23.3
 -23.2 -23.1 -23.  -22.9 -22.8 -22.7 -22.6 -22.5 -22.3 -22.2 -22.  -21.9
 -21.8 -21.7 -21.6 -21.5 -21.4 -21.3 -21.2 -21.1 -20.9 -20.7 -20.6 -20.5
 -20.4 -20.3 -20.2 -20.1 -20.  -19.9 -19.8 -19.7 -19.6 -19.5 -19.4 -19.3
 -19.2 -19.1 -19.  -18.9 -18.8 -18.7 -18.6 -18.5 -18.4 -18.3 -18.2 -18.1
 -18.  -17.9 -17.8 -17.7 -17.6 -17.5 -17.4 -17.3 -17.2 -17.1 -17.  -16.9
 -16.8 -16.7 -16.6 -16.5 -16.4 -16.3 -16.2 -16.1 -16.  -15.9 -15.8 -15.7
 -15.6 -15.5 -15.4 -15.3 -15.2 -15.1 -15.  -14.9 -14.8 -14.7 -14.6 -14.5
 -14.4 -14.3 -14.2 -14.1 -14.  -13.9 -13.8 -13.7 -13.6 -13.5 -13.4 -13.3
 -13.2 -13.1 -13.  -12.9 -12.8 -12.7 -12.6 -12.5 -12.4 -12.3 -12.2 -12.1
 -12.  -11.9 -11.8 -11.7 -11.6 -11.5 -11.4 -11.3

### How many values are there for AvgTemperature and what are they

In [33]:
print(df['AvgTemperature'].nunique())
snow = df['AvgTemperature'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

179
[-2.  -1.  -0.5  0.   0.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.
  6.5  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.
 13.5 14.  14.5 15.  15.5 16.  16.5 17.  17.5 18.  18.5 19.  19.5 20.
 20.5 21.  21.5 22.  22.5 23.  23.5 24.  24.5 25.  25.5 26.  26.5 27.
 27.5 28.  28.5 29.  29.5 30.  30.5 31.  31.5 32.  32.5 33.  33.5 34.
 34.5 35.  35.5 36.  36.5 37.  37.5 38.  38.5 39.  39.5 40.  40.5 41.
 41.5 42.  42.5 43.  43.5 44.  44.5 45.  45.5 46.  46.5 47.  47.5 48.
 48.5 49.  49.5 50.  50.5 51.  51.5 52.  52.5 53.  53.5 54.  54.5 55.
 55.5 56.  56.5 57.  57.5 58.  58.5 59.  59.5 60.  60.5 61.  61.5 62.
 62.5 63.  63.5 64.  64.5 65.  65.5 66.  66.5 67.  67.5 68.  68.5 69.
 69.5 70.  70.5 71.  71.5 72.  72.5 73.  73.5 74.  74.5 75.  75.5 76.
 76.5 77.  77.5 78.  78.5 79.  79.5 80.  80.5 81.  81.5 82.  82.5 83.
 83.5 84.  84.5 85.  85.5 86.  86.5 87.  87.5 88.  88.5]


### How many values are there for AvgTemperatureNormal and what are they

In [34]:
print(df['AvgTemperatureNormal'].nunique())
snow = df['AvgTemperatureNormal'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

340
[25.6 25.7 25.8 25.9 26.  26.1 26.2 26.3 26.4 26.5 26.6 26.8 26.9 27.1
 27.2 27.3 27.4 27.5 27.7 27.8 27.9 28.  28.1 28.2 28.3 28.5 28.7 28.8
 28.9 29.1 29.3 29.5 29.6 29.7 29.9 30.  30.1 30.3 30.4 30.6 30.8 30.9
 31.  31.3 31.5 31.6 31.8 32.  32.3 32.4 32.5 32.7 32.8 32.9 33.1 33.3
 33.5 33.6 33.8 33.9 34.1 34.2 34.4 34.6 34.7 35.  35.3 35.4 35.6 35.8
 35.9 36.2 36.3 36.5 36.6 36.9 37.2 37.3 37.6 37.7 37.9 38.1 38.3 38.5
 38.6 38.8 39.  39.2 39.4 39.6 39.7 39.9 40.1 40.3 40.5 40.7 40.9 41.
 41.3 41.4 41.7 42.  42.1 42.4 42.5 42.7 42.9 43.  43.3 43.4 43.7 44.
 44.1 44.3 44.5 44.6 44.9 45.2 45.3 45.5 45.7 45.8 46.1 46.2 46.4 46.6
 46.7 47.  47.3 47.4 47.6 47.8 47.9 48.2 48.5 48.6 48.7 49.  49.3 49.4
 49.7 49.8 50.  50.2 50.3 50.6 50.9 51.2 51.3 51.6 51.7 51.9 52.  52.2
 52.4 52.6 52.7 52.9 53.1 53.3 53.4 53.6 53.7 53.8 53.9 54.  54.1 54.3
 54.4 54.7 54.8 54.9 55.1 55.2 55.3 55.4 55.5 55.6 55.7 55.8 56.  56.2
 56.3 56.4 56.6 56.8 56.9 57.  57.2 57.3 57.4 57.5 57.6 57.7 57.8 57.9
 58.

### How many values are there for AvgTemperatureDeparture and what are they

In [35]:
print(df['AvgTemperatureDeparture'].nunique())
snow = df['AvgTemperatureDeparture'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

502
[-30.5 -28.4 -27.2 -26.6 -26.5 -26.3 -26.2 -26.  -25.9 -25.7 -25.3 -24.7
 -24.4 -24.3 -23.9 -23.8 -23.6 -23.4 -23.2 -23.1 -22.9 -22.8 -22.6 -22.5
 -22.4 -22.2 -22.1 -22.  -21.9 -21.8 -21.7 -21.6 -21.5 -21.3 -21.2 -21.1
 -20.9 -20.7 -20.6 -20.5 -20.4 -20.3 -20.2 -20.1 -20.  -19.9 -19.8 -19.7
 -19.6 -19.5 -19.4 -19.3 -19.2 -19.1 -19.  -18.9 -18.8 -18.7 -18.6 -18.5
 -18.4 -18.3 -18.2 -18.1 -18.  -17.9 -17.8 -17.6 -17.5 -17.4 -17.3 -17.2
 -17.1 -17.  -16.9 -16.8 -16.7 -16.6 -16.5 -16.4 -16.3 -16.2 -16.1 -16.
 -15.9 -15.8 -15.7 -15.6 -15.5 -15.4 -15.3 -15.2 -15.1 -15.  -14.9 -14.8
 -14.7 -14.6 -14.5 -14.4 -14.3 -14.2 -14.1 -14.  -13.9 -13.8 -13.7 -13.6
 -13.5 -13.4 -13.3 -13.2 -13.1 -13.  -12.9 -12.8 -12.7 -12.6 -12.5 -12.4
 -12.3 -12.2 -12.1 -12.  -11.9 -11.8 -11.7 -11.6 -11.5 -11.4 -11.3 -11.2
 -11.1 -11.  -10.9 -10.8 -10.7 -10.6 -10.5 -10.4 -10.3 -10.2 -10.1 -10.
  -9.9  -9.8  -9.7  -9.6  -9.5  -9.4  -9.3  -9.2  -9.1  -9.   -8.9  -8.8
  -8.7  -8.6  -8.5  -8.4  -8.3  -8.2  -8.1  -8.  

### How many values are there for AtObsTemperature and what are they

In [36]:
print(df['AtObsTemperature'].nunique())
snow = df['AtObsTemperature'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

1
['M']


### How many values are there for Precipitation and what are they

### After adding the 12.2.2020 to 12.27.2020 data I had to convert it to a string

In [37]:
df[['Precipitation']] = df[['Precipitation']].astype(str)

In [38]:
print(df['Precipitation'].nunique())
snow = df['Precipitation'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

264
['0.0' '0.001' '0.01' '0.02' '0.03' '0.04' '0.05' '0.06' '0.07' '0.08'
 '0.09' '0.1' '0.11' '0.12' '0.13' '0.14' '0.15' '0.16' '0.17' '0.18'
 '0.19' '0.2' '0.21' '0.22' '0.23' '0.24' '0.25' '0.26' '0.27' '0.28'
 '0.29' '0.3' '0.31' '0.32' '0.33' '0.34' '0.35' '0.36' '0.37' '0.38'
 '0.39' '0.4' '0.41' '0.42' '0.43' '0.44' '0.45' '0.46' '0.47' '0.48'
 '0.49' '0.5' '0.51' '0.52' '0.53' '0.54' '0.55' '0.56' '0.57' '0.58'
 '0.59' '0.6' '0.61' '0.62' '0.63' '0.64' '0.65' '0.66' '0.67' '0.68'
 '0.69' '0.7' '0.71' '0.72' '0.73' '0.74' '0.75' '0.76' '0.77' '0.78'
 '0.79' '0.8' '0.81' '0.82' '0.83' '0.84' '0.85' '0.86' '0.87' '0.88'
 '0.89' '0.9' '0.91' '0.92' '0.93' '0.94' '0.95' '0.96' '0.97' '0.98'
 '0.99' '1.0' '1.01' '1.02' '1.03' '1.04' '1.05' '1.06' '1.07' '1.08'
 '1.09' '1.1' '1.11' '1.12' '1.13' '1.14' '1.15' '1.16' '1.17' '1.18'
 '1.19' '1.2' '1.21' '1.22' '1.23' '1.24' '1.25' '1.26' '1.27' '1.28'
 '1.29' '1.3' '1.31' '1.32' '1.33' '1.34' '1.35' '1.36' '1.37' '1.38'
 '1.39' '1.4' '

### How many values are there for PrecipitationNormal and what are they

### After adding the 12.2.2020 to 12.27.2020 data I had to convert it to a string

In [39]:
df[['PrecipitationNormal']] = df[['PrecipitationNormal']].astype(str)

In [40]:
print(df['PrecipitationNormal'].nunique())
snow = df['PrecipitationNormal'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

11
['0.09' '0.1' '0.11' '0.12' '0.13' '0.14' '0.15' '0.16' '0.17' '0.18' 'M']


### How many values are there for PrecipitationDeparture and what are they

### After adding the 12.2.2020 to 12.27.2020 data I had to convert it to a string

In [41]:
df[['PrecipitationDeparture']] = df[['PrecipitationDeparture']].astype(str)

In [42]:
print(df['PrecipitationDeparture'].nunique())
snow = df['PrecipitationDeparture'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

268
['-0.01' '-0.02' '-0.03' '-0.04' '-0.05' '-0.06' '-0.07' '-0.08' '-0.09'
 '-0.1' '-0.11' '-0.12' '-0.13' '-0.14' '-0.15' '-0.16' '-0.17' '-0.18'
 '0' '0.01' '0.02' '0.03' '0.04' '0.05' '0.06' '0.07' '0.08' '0.09' '0.1'
 '0.11' '0.12' '0.13' '0.14' '0.15' '0.16' '0.17' '0.18' '0.19' '0.2'
 '0.21' '0.22' '0.23' '0.24' '0.25' '0.26' '0.27' '0.28' '0.29' '0.3'
 '0.31' '0.32' '0.33' '0.34' '0.35' '0.36' '0.37' '0.38' '0.39' '0.4'
 '0.41' '0.42' '0.43' '0.44' '0.45' '0.46' '0.47' '0.48' '0.49' '0.5'
 '0.51' '0.52' '0.53' '0.54' '0.55' '0.56' '0.57' '0.58' '0.59' '0.6'
 '0.61' '0.62' '0.63' '0.64' '0.65' '0.66' '0.67' '0.68' '0.69' '0.7'
 '0.71' '0.72' '0.73' '0.74' '0.75' '0.76' '0.77' '0.78' '0.79' '0.8'
 '0.81' '0.82' '0.83' '0.84' '0.85' '0.86' '0.87' '0.88' '0.89' '0.9'
 '0.91' '0.92' '0.93' '0.94' '0.95' '0.96' '0.97' '0.98' '0.99' '1' '1.01'
 '1.02' '1.03' '1.04' '1.05' '1.06' '1.07' '1.08' '1.09' '1.1' '1.11'
 '1.12' '1.13' '1.14' '1.15' '1.16' '1.17' '1.18' '1.19' '1.2' '1.21'
 '

### How many values are there for Snowfall and what are they

In [43]:
print(df['Snowfall'].nunique())
snow = df['Snowfall'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

115
[0.00e+00 1.00e-02 5.00e-02 1.00e-01 2.00e-01 3.00e-01 4.00e-01 5.00e-01
 6.00e-01 7.00e-01 8.00e-01 9.00e-01 1.00e+00 1.10e+00 1.20e+00 1.30e+00
 1.40e+00 1.50e+00 1.60e+00 1.70e+00 1.80e+00 1.90e+00 2.00e+00 2.10e+00
 2.20e+00 2.30e+00 2.40e+00 2.50e+00 2.60e+00 2.70e+00 2.80e+00 2.90e+00
 3.00e+00 3.10e+00 3.20e+00 3.30e+00 3.40e+00 3.50e+00 3.60e+00 3.70e+00
 3.80e+00 3.90e+00 4.00e+00 4.10e+00 4.20e+00 4.40e+00 4.50e+00 4.60e+00
 4.70e+00 4.80e+00 4.90e+00 5.00e+00 5.10e+00 5.20e+00 5.30e+00 5.50e+00
 5.60e+00 5.70e+00 5.80e+00 5.90e+00 6.00e+00 6.10e+00 6.20e+00 6.30e+00
 6.40e+00 6.50e+00 6.60e+00 6.70e+00 6.80e+00 6.90e+00 7.00e+00 7.10e+00
 7.20e+00 7.30e+00 7.40e+00 7.50e+00 7.60e+00 7.80e+00 7.90e+00 8.00e+00
 8.10e+00 8.40e+00 8.60e+00 8.70e+00 8.80e+00 8.90e+00 9.00e+00 9.10e+00
 9.30e+00 9.50e+00 9.70e+00 9.90e+00 1.00e+01 1.02e+01 1.03e+01 1.04e+01
 1.05e+01 1.06e+01 1.08e+01 1.09e+01 1.13e+01 1.14e+01 1.17e+01 1.19e+01
 1.23e+01 1.25e+01 1.34e+01 1.39e+01 1.41e+01 1

### How many values are there for SnowfallNormal and what are they

### After adding the 12.2.2020 to 12.27.2020 data I had to convert it to a string

In [44]:
df[['SnowfallNormal']] = df[['SnowfallNormal']].astype(str)

In [45]:
print(df['SnowfallNormal'].nunique())
snow = df['SnowfallNormal'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

7
['0' '0.1' '0.2' '0.3' '0.4' '0.5' 'M']


### How many values are there for SnowfallDeparture and what are they

### After adding the 12.2.2020 to 12.27.2020 data I had to convert it to a string

In [46]:
df[['SnowfallDeparture']] = df[['SnowfallDeparture']].astype(str)

In [47]:
print(df['SnowfallDeparture'].nunique())
snow = df['SnowfallDeparture'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

112
['-0.1' '-0.2' '-0.3' '-0.4' '-0.5' '0' '0.1' '0.2' '0.3' '0.4' '0.5'
 '0.6' '0.7' '0.8' '0.9' '1' '1.1' '1.2' '1.3' '1.4' '1.5' '1.6' '1.7'
 '1.8' '1.9' '10' '10.3' '10.4' '11' '11.3' '11.5' '12.1' '12.2' '12.3'
 '13' '14' '14.6' '15.1' '15.6' '15.9' '2' '2.1' '2.2' '2.3' '2.4' '2.5'
 '2.6' '2.7' '2.8' '2.9' '21.5' '23.6' '3' '3.1' '3.2' '3.4' '3.5' '3.6'
 '3.7' '3.8' '3.9' '4' '4.1' '4.2' '4.3' '4.4' '4.5' '4.6' '4.7' '4.8'
 '4.9' '5' '5.1' '5.2' '5.3' '5.4' '5.5' '5.6' '5.7' '5.8' '5.9' '6' '6.1'
 '6.2' '6.3' '6.4' '6.5' '6.6' '6.7' '6.8' '6.9' '7' '7.1' '7.4' '7.5'
 '7.6' '7.7' '7.8' '8' '8.2' '8.3' '8.5' '8.6' '8.7' '8.8' '9.1' '9.3'
 '9.4' '9.6' '9.8' '9.9' 'M']


### How many values are there for SnowDepth and what are they

In [48]:
print(df['SnowDepth'].nunique())
snow = df['SnowDepth'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

34
['0' '1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21'
 '22' '23' '24' '25' '26' '28' '29' '3' '33' '34' '35' '38' '4' '5' '6'
 '7' '8' '9' 'M']


### How many values are there for HDD and what are they

In [49]:
print(df['HDD'].nunique())
snow = df['HDD'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

68
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67.]


### How many values are there for HDDNormal and what are they

In [50]:
print(df['HDDNormal'].nunique())
snow = df['HDDNormal'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

114
[ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1  1.2  1.3
  1.4  1.5  1.6  1.7  1.8  1.9  2.   2.2  2.4  2.6  2.8  2.9  3.   3.1
  3.2  3.3  3.4  3.5  3.6  3.7  3.9  4.   4.1  4.2  4.3  4.5  4.6  4.7
  4.8  4.9  5.   5.1  5.2  5.3  5.4  5.6  5.7  5.9  6.   6.2  6.4  6.5
  6.6  6.8  6.9  7.   7.1  7.2  7.3  7.5  7.6  7.9  8.   8.1  8.2  8.4
  8.5  8.7  8.9  9.   9.2  9.3  9.6  9.9 10.  10.2 10.3 10.5 10.6 11.
 11.3 11.6 12.  13.  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.
 24.  25.  26.  27.  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.
 38.  39. ]


### How many values are there for HDDDeparture and what are they

In [51]:
print(df['HDDDeparture'].nunique())
snow = df['HDDDeparture'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

131
[-35.  -34.  -32.  -31.  -29.  -28.  -27.  -26.  -25.  -24.  -23.  -22.
 -21.  -20.  -19.  -18.  -17.  -16.  -15.  -14.  -13.  -12.  -11.6 -11.
 -10.   -9.   -8.6  -8.   -7.6  -7.   -6.3  -6.   -5.9  -5.6  -5.4  -5.3
  -5.2  -5.1  -5.   -4.9  -4.8  -4.7  -4.5  -4.3  -4.1  -4.   -3.9  -3.6
  -3.5  -3.3  -3.1  -3.   -2.9  -2.7  -2.6  -2.5  -2.4  -2.3  -2.2  -2.
  -1.9  -1.8  -1.7  -1.6  -1.5  -1.4  -1.3  -1.2  -1.1  -1.   -0.9  -0.8
  -0.7  -0.6  -0.5  -0.4  -0.3  -0.2  -0.1   0.    0.5   0.6   0.7   0.8
   1.    1.2   1.4   1.5   1.7   1.9   2.    2.2   2.3   2.7   3.    3.4
   3.8   3.9   4.    4.1   5.    5.1   5.6   5.8   6.    7.    7.1   8.
   9.   10.   11.   12.   13.   13.5  14.   14.3  15.   16.   17.   18.
  19.   20.   21.   22.   23.   24.   25.   26.   27.   28.   30. ]


### How many values are there for CDD and what are they

In [52]:
print(df['CDD'].nunique())
snow = df['CDD'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

25
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24.]


### How many values are there for CDDNormal and what are they

In [53]:
print(df['CDDNormal'].nunique())
snow = df['CDDNormal'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

90
[0.  0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8
 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6 2.7 2.8 2.9 3.  3.1 3.2 3.3 3.4 3.5 3.6
 3.7 3.8 3.9 4.  4.1 4.3 4.5 4.7 4.8 4.9 5.  5.1 5.2 5.3 5.5 5.7 5.8 5.9
 6.  6.1 6.2 6.3 6.4 6.5 6.6 6.7 6.9 7.  7.2 7.3 7.4 7.5 7.6 7.7 7.8 7.9
 8.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8 8.9 9.  9.1 9.2 9.3 9.4 9.5 9.6 9.7]


### How many values are there for CDDDeparture and what are they

In [54]:
print(df['CDDDeparture'].nunique())
snow = df['CDDDeparture'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

120
[-9.3 -9.  -8.5 -8.4 -8.1 -8.  -7.7 -7.2 -7.  -6.7 -6.5 -6.  -5.8 -5.6
 -5.5 -5.3 -5.2 -5.1 -5.  -4.8 -4.5 -4.2 -4.  -3.9 -3.7 -3.6 -3.4 -3.3
 -3.1 -3.  -2.9 -2.7 -2.6 -2.4 -2.3 -2.2 -2.1 -2.  -1.9 -1.8 -1.7 -1.5
 -1.4 -1.3 -1.2 -1.1 -1.  -0.9 -0.8 -0.7 -0.6 -0.5 -0.4 -0.3 -0.2 -0.1
  0.   0.2  0.5  0.7  0.8  1.   1.3  1.5  1.7  2.   2.3  2.4  2.5  2.7
  2.8  2.9  3.   3.2  3.3  3.4  3.5  3.7  4.   4.2  4.4  4.7  5.   5.3
  5.5  5.7  5.9  6.   6.2  6.3  6.4  6.8  7.   7.3  7.4  7.5  7.9  8.
  8.3  8.6  8.9  9.   9.2  9.3  9.6  9.7  9.8 10.  10.7 10.8 11.  11.4
 12.  13.  13.5 13.7 14.  14.2 15.  16. ]


### How many values are there for GDD and what are they

In [55]:
print(df['GDD'].nunique())
snow = df['GDD'].unique()
snow_sort = np.sort(snow)
print(snow_sort)

40
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39.]


### Some of the data is 'M' and 'T'
* T: Trace amount- Less than 0.01" precipitation; less than 0.1" snowfall; less than 1" snow depth.

* AtObsTemperature
* PrecipitationNormal
* PrecipitationDeparture
* SnowfallNormal
* SnowfallDeparture
* SnowDepth

In [56]:
atobstemperature = df.loc[df['AtObsTemperature'] == 'T']
print(len(atobstemperature))
atobstemperature = df.loc[df['AtObsTemperature'] == 'M']
print(len(atobstemperature))

0
15263


In [57]:
precipitationnormal = df.loc[df['PrecipitationNormal'] == 'T']
print(len(precipitationnormal))
precipitationnormal = df.loc[df['PrecipitationNormal'] == 'M']
print(len(precipitationnormal))

0
11


In [58]:
precipitationdeparture = df.loc[df['PrecipitationDeparture'] == 'T']
print(len(precipitationdeparture))
precipitationdeparture = df.loc[df['PrecipitationDeparture'] == 'M']
print(len(precipitationdeparture))

0
11


In [59]:
snowfallnormal = df.loc[df['SnowfallNormal'] == 'T']
print(len(snowfallnormal))
snowfallnormal = df.loc[df['SnowfallNormal'] == 'M']
print(len(snowfallnormal))

0
11


In [60]:
snowfalldeparture = df.loc[df['SnowfallDeparture'] == 'T']
print(len(snowfalldeparture))
snowfalldeparture = df.loc[df['SnowfallDeparture'] == 'M']
print(len(snowfalldeparture))

0
2222


In [61]:
snowdepth = df.loc[df['SnowDepth'] == 'T']
print(len(snowdepth))
snowdepth = df.loc[df['SnowDepth'] == 'M']
print(len(snowdepth))

0
2188


In [62]:
snowfall_t = df.loc[df['Snowfall'] == 'T']
print(len(snowfall_t))
snowfall_m = df.loc[df['Snowfall'] == 'M']
print(len(snowfall_m))

0
0


In [63]:
precip_t = df.loc[df['Precipitation'] == 'T']
print(len(precip_t))
precip_m = df.loc[df['Precipitation'] == 'M']
print(len(precip_m))

0
0


# Drop AtObsTemperature

In [64]:
df = df.drop(['AtObsTemperature'], axis = 1)

# PrecipitationNormal, PrecipitationDeparture and SnowfallNormal
* These are all 'M' because they are 2/29

In [65]:
precipitationnormal = df.loc[df['PrecipitationNormal'] == 'M']
print(len(precipitationnormal))

11


In [66]:
#df.loc[df['PrecipitationNormal'] == 'M'].unique()
df_precip_normal = df[(df['PrecipitationNormal'] == 'M')]
df_precip_normal[['Date','Precipitation','PrecipitationNormal','PrecipitationDeparture','Snowfall','SnowfallNormal']]

,Date,Precipitation,PrecipitationNormal,PrecipitationDeparture,Snowfall,SnowfallNormal
59,2/29/80,0.0,M,M,0.0,M
1520,2/29/84,0.05,M,M,0.3,M
2981,2/29/88,0.0,M,M,0.0,M
4442,2/29/92,0.02,M,M,0.0,M
5903,2/29/96,0.0,M,M,0.0,M
7364,2/29/00,0.0,M,M,0.0,M
8825,2/29/04,0.0,M,M,0.0,M
10286,2/29/08,0.05,M,M,0.8,M
11747,2/29/12,0.59,M,M,4.4,M
13208,2/29/16,0.02,M,M,0.0,M


### For the leap year 2/29 days, just make PrecipitationNormal and PrecipitationDeparture equal to Precipitation and SnowfallNormal equal to Snowfall

In [67]:
df.loc[(df['Year'] == 1980) & (df['Month'] == 2) & (df['Day'] == 29), 'SnowfallNormal'] = '0.0'
df.loc[(df['Year'] == 1984) & (df['Month'] == 2) & (df['Day'] == 29), 'SnowfallNormal'] = '0.3'
df.loc[(df['Year'] == 1988) & (df['Month'] == 2) & (df['Day'] == 29), 'SnowfallNormal'] = '0.0'
df.loc[(df['Year'] == 1992) & (df['Month'] == 2) & (df['Day'] == 29), 'SnowfallNormal'] = '0.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 2) & (df['Day'] == 29), 'SnowfallNormal'] = '0.0'
df.loc[(df['Year'] == 2000) & (df['Month'] == 2) & (df['Day'] == 29), 'SnowfallNormal'] = '0.0'
df.loc[(df['Year'] == 2004) & (df['Month'] == 2) & (df['Day'] == 29), 'SnowfallNormal'] = '0.0'
df.loc[(df['Year'] == 2008) & (df['Month'] == 2) & (df['Day'] == 29), 'SnowfallNormal'] = '0.08'
df.loc[(df['Year'] == 2012) & (df['Month'] == 2) & (df['Day'] == 29), 'SnowfallNormal'] = '4.4'
df.loc[(df['Year'] == 2016) & (df['Month'] == 2) & (df['Day'] == 29), 'SnowfallNormal'] = '0.0'
df.loc[(df['Year'] == 2020) & (df['Month'] == 2) & (df['Day'] == 29), 'SnowfallNormal'] = '0.0'

In [68]:
df.loc[(df['Year'] == 1980) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationNormal'] = '0.0'
df.loc[(df['Year'] == 1984) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationNormal'] = '0.05'
df.loc[(df['Year'] == 1988) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationNormal'] = '0.0'
df.loc[(df['Year'] == 1992) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationNormal'] = '0.02'
df.loc[(df['Year'] == 1996) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationNormal'] = '0.0'
df.loc[(df['Year'] == 2000) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationNormal'] = '0.0'
df.loc[(df['Year'] == 2004) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationNormal'] = '0.0'
df.loc[(df['Year'] == 2008) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationNormal'] = '0.05'
df.loc[(df['Year'] == 2012) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationNormal'] = '0.59'
df.loc[(df['Year'] == 2016) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationNormal'] = '0.02'
df.loc[(df['Year'] == 2020) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationNormal'] = '0.0'

In [69]:
df.loc[(df['Year'] == 1980) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationDeparture'] = '0.0'
df.loc[(df['Year'] == 1984) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationDeparture'] = '0.05'
df.loc[(df['Year'] == 1988) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationDeparture'] = '0.0'
df.loc[(df['Year'] == 1992) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationDeparture'] = '0.02'
df.loc[(df['Year'] == 1996) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationDeparture'] = '0.0'
df.loc[(df['Year'] == 2000) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationDeparture'] = '0.0'
df.loc[(df['Year'] == 2004) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationDeparture'] = '0.0'
df.loc[(df['Year'] == 2008) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationDeparture'] = '0.05'
df.loc[(df['Year'] == 2012) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationDeparture'] = '0.59'
df.loc[(df['Year'] == 2016) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationDeparture'] = '0.02'
df.loc[(df['Year'] == 2020) & (df['Month'] == 2) & (df['Day'] == 29), 'PrecipitationDeparture'] = '0.0'

In [70]:
precipitationdeparture = df.loc[df['PrecipitationDeparture'] == 'T']
print(len(precipitationdeparture))
precipitationdeparture = df.loc[df['PrecipitationDeparture'] == 'M']
print(len(precipitationdeparture))

0
0


In [71]:
precipitationdeparture

,Date,MaxTemperature,MaxTemperatureNormal,MaxTemperatureDeparture,MinTemperature,MinTemperatureNormal,MinTemperatureDeparture,AvgTemperature,AvgTemperatureNormal,AvgTemperatureDeparture,...,HDDNormal,HDDDeparture,CDD,CDDNormal,CDDDeparture,GDD,Precipitation,Year,Month,Day


## If Precipitation is 0 then make Snowfall 0

In [72]:
snowfall = df['Snowfall'].unique()
snowfall_sort = np.sort(snowfall)
print(snowfall_sort)

[0.00e+00 1.00e-02 5.00e-02 1.00e-01 2.00e-01 3.00e-01 4.00e-01 5.00e-01
 6.00e-01 7.00e-01 8.00e-01 9.00e-01 1.00e+00 1.10e+00 1.20e+00 1.30e+00
 1.40e+00 1.50e+00 1.60e+00 1.70e+00 1.80e+00 1.90e+00 2.00e+00 2.10e+00
 2.20e+00 2.30e+00 2.40e+00 2.50e+00 2.60e+00 2.70e+00 2.80e+00 2.90e+00
 3.00e+00 3.10e+00 3.20e+00 3.30e+00 3.40e+00 3.50e+00 3.60e+00 3.70e+00
 3.80e+00 3.90e+00 4.00e+00 4.10e+00 4.20e+00 4.40e+00 4.50e+00 4.60e+00
 4.70e+00 4.80e+00 4.90e+00 5.00e+00 5.10e+00 5.20e+00 5.30e+00 5.50e+00
 5.60e+00 5.70e+00 5.80e+00 5.90e+00 6.00e+00 6.10e+00 6.20e+00 6.30e+00
 6.40e+00 6.50e+00 6.60e+00 6.70e+00 6.80e+00 6.90e+00 7.00e+00 7.10e+00
 7.20e+00 7.30e+00 7.40e+00 7.50e+00 7.60e+00 7.80e+00 7.90e+00 8.00e+00
 8.10e+00 8.40e+00 8.60e+00 8.70e+00 8.80e+00 8.90e+00 9.00e+00 9.10e+00
 9.30e+00 9.50e+00 9.70e+00 9.90e+00 1.00e+01 1.02e+01 1.03e+01 1.04e+01
 1.05e+01 1.06e+01 1.08e+01 1.09e+01 1.13e+01 1.14e+01 1.17e+01 1.19e+01
 1.23e+01 1.25e+01 1.34e+01 1.39e+01 1.41e+01 1.50e

In [73]:
snowfall_t = df.loc[df['Snowfall'] == 'T']
print(len(snowfall_t))
snowfall_m = df.loc[df['Snowfall'] == 'M']
print(len(snowfall_m))

0
0


### How many T and M in SnowDepth

In [74]:
snowdepth_t = df.loc[df['SnowDepth'] == 'T']
print(len(snowdepth_t))
snowdepth_m = df.loc[df['SnowDepth'] == 'M']
print(len(snowdepth_m))

0
2188


### How many T and M in Precipitation

In [75]:
precip_t = df.loc[df['Precipitation'] == 'T']
print(len(precip_t))
precip_m = df.loc[df['Precipitation'] == 'M']
print(len(precip_m))

0
0


### Convert Precipitation back to float as there are no T or M

In [76]:
df[['Precipitation']] = df[['Precipitation']].astype(float)

### If it May, June, July, August or September then set SnowDepth to 0.0

In [77]:
#df.loc[df['Month'].eq('5') | df['Month'].eq('6') | df['Month'].eq('7') | df['Month'].eq('8') | df['Month'].eq('9'), 'SnowDepth'] = '0.0'
df.loc[df['Month'].eq(5) | df['Month'].eq(6) | df['Month'].eq(7) | df['Month'].eq(8) | df['Month'].eq(9), 'SnowDepth'] = '0.0'

### How many T and M for SnowDepth

In [78]:
snowdepth_t2 = df.loc[df['SnowDepth'] == 'T']
print(len(snowdepth_t2))
snowdepth_m2 = df.loc[df['SnowDepth'] == 'M']
print(len(snowdepth_m2))

0
2188


In [79]:
#df_summer = df.loc[df['SnowDepth'] == 'M']
df_summer = df[(df['Month'] > 4) & (df['Month'] < 10) & (df['SnowDepth'] == 'M')]
df_summer[['Date','AvgTemperature', 'MinTemperature', 'MaxTemperature','Precipitation', 'Snowfall', 'SnowDepth', 'Month']]

,Date,AvgTemperature,MinTemperature,MaxTemperature,Precipitation,Snowfall,SnowDepth,Month


### Only looking at SnowDepth

In [80]:
snow_m = df.loc[df['SnowDepth'] == 'M']
snow_m['Precipitation']

6118     0.00
6119     0.07
6120     0.01
6121     0.00
6122     0.00
         ... 
13601    0.46
13602    0.00
13603    0.01
13604    1.03
14969    2.12
Name: Precipitation, Length: 2188, dtype: float64

In [81]:
snow_m[['Date','AvgTemperature','Precipitation', 'Snowfall', 'SnowDepth']]

,Date,AvgTemperature,Precipitation,Snowfall,SnowDepth
6118,10/1/96,57.0,0.00,0.0,M
6119,10/2/96,58.5,0.07,0.0,M
6120,10/3/96,49.5,0.01,0.0,M
6121,10/4/96,41.5,0.00,0.0,M
6122,10/5/96,42.5,0.00,0.0,M
...,...,...,...,...,...
13601,3/28/17,40.5,0.46,0.0,M
13602,3/29/17,43.0,0.00,0.0,M
13603,3/30/17,42.0,0.01,0.0,M
13604,3/31/17,36.5,1.03,1.0,M


### Average Temperature Greater than 32

In [82]:
snow_m_greater_than_32 = snow_m.loc[((snow_m['AvgTemperature'] > 32))]

### Minimum Temperature Greater than 32

In [83]:
snow_m_min_greater_than_32 = snow_m.loc[((snow_m['MinTemperature'] > 32))]
snow_m_min_greater_than_32

,Date,MaxTemperature,MaxTemperatureNormal,MaxTemperatureDeparture,MinTemperature,MinTemperatureNormal,MinTemperatureDeparture,AvgTemperature,AvgTemperatureNormal,AvgTemperatureDeparture,...,HDDNormal,HDDDeparture,CDD,CDDNormal,CDDDeparture,GDD,Precipitation,Year,Month,Day
6118,10/1/96,72.0,68.9,3.1,42.0,46.3,-4.3,57.0,57.6,-0.6,...,8.0,0.0,0.0,1.0,-1.0,7.0,0.00,1996,10,1
6119,10/2/96,69.0,68.5,0.5,48.0,45.9,2.1,58.5,57.2,1.3,...,8.0,-2.0,0.0,1.0,-1.0,9.0,0.07,1996,10,2
6120,10/3/96,63.0,68.1,-5.1,36.0,45.5,-9.5,49.5,56.8,-7.3,...,9.0,6.0,0.0,1.0,-1.0,0.0,0.01,1996,10,3
6124,10/7/96,67.0,66.5,0.5,33.0,43.9,-10.9,50.0,55.2,-5.2,...,10.0,5.0,0.0,0.0,0.0,0.0,0.00,1996,10,7
6125,10/8/96,59.0,66.1,-7.1,51.0,43.5,7.5,55.0,54.8,0.2,...,11.0,-1.0,0.0,0.0,0.0,5.0,1.33,1996,10,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13600,3/27/17,42.0,52.0,-10.0,33.0,31.4,1.6,37.5,41.7,-4.2,...,23.0,4.0,0.0,0.0,0.0,0.0,0.40,2017,3,27
13601,3/28/17,44.0,52.4,-8.4,37.0,31.7,5.3,40.5,42.1,-1.6,...,23.0,1.0,0.0,0.0,0.0,0.0,0.46,2017,3,28
13602,3/29/17,53.0,52.8,0.2,33.0,32.1,0.9,43.0,42.5,0.5,...,23.0,-1.0,0.0,0.0,0.0,0.0,0.00,2017,3,29
13604,3/31/17,40.0,53.7,-13.7,33.0,32.8,0.2,36.5,43.3,-6.8,...,22.0,6.0,0.0,0.0,0.0,0.0,1.03,2017,3,31


### Minimum temperatures is greater than 32.
* Precipitation T or M

In [84]:
m1 = snow_m_min_greater_than_32.loc[snow_m_min_greater_than_32['Precipitation'] == 'M']
m1

,Date,MaxTemperature,MaxTemperatureNormal,MaxTemperatureDeparture,MinTemperature,MinTemperatureNormal,MinTemperatureDeparture,AvgTemperature,AvgTemperatureNormal,AvgTemperatureDeparture,...,HDDNormal,HDDDeparture,CDD,CDDNormal,CDDDeparture,GDD,Precipitation,Year,Month,Day


In [85]:
t1 = snow_m_min_greater_than_32.loc[snow_m_min_greater_than_32['Precipitation'] == 'T']
t1

,Date,MaxTemperature,MaxTemperatureNormal,MaxTemperatureDeparture,MinTemperature,MinTemperatureNormal,MinTemperatureDeparture,AvgTemperature,AvgTemperatureNormal,AvgTemperatureDeparture,...,HDDNormal,HDDDeparture,CDD,CDDNormal,CDDDeparture,GDD,Precipitation,Year,Month,Day


In [87]:
print(snow_m_min_greater_than_32['Precipitation'].nunique())
p = snow_m_min_greater_than_32['Precipitation'].unique()
p_sort = np.sort(p)
print(p_sort)

120
[0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13
 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27
 0.28 0.29 0.31 0.33 0.34 0.35 0.36 0.37 0.39 0.4  0.41 0.42 0.43 0.44
 0.45 0.46 0.48 0.49 0.5  0.51 0.52 0.54 0.55 0.56 0.58 0.59 0.61 0.63
 0.64 0.65 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.75 0.76 0.77 0.78 0.79
 0.8  0.83 0.84 0.86 0.88 0.89 0.9  0.91 0.92 0.94 0.95 0.96 0.98 1.03
 1.07 1.08 1.1  1.11 1.12 1.14 1.16 1.17 1.18 1.2  1.24 1.25 1.26 1.27
 1.33 1.39 1.42 1.48 1.49 1.52 1.57 1.66 1.67 1.7  1.82 1.84 1.96 1.99
 2.05 2.07 2.11 2.12 2.22 2.4  4.41 5.26]


In [88]:
m2 = snow_m_min_greater_than_32.loc[snow_m_min_greater_than_32['Snowfall'] == 'M']
# 589 missing values
# Check if Precipitation is greater than .5
p1 = snow_m_min_greater_than_32.loc[((snow_m_min_greater_than_32['Precipitation'] > 1))]
dates = []
for i in p1['Date']:
    dates.append(i)
print(dates)

['10/8/96', '10/20/96', '11/9/96', '11/1/97', '3/9/98', '3/19/98', '10/8/98', '10/14/98', '11/26/98', '3/22/99', '11/2/99', '11/27/99', '1/10/00', '4/21/00', '3/3/02', '4/28/02', '10/11/02', '10/12/02', '10/15/03', '10/27/03', '10/29/03', '11/20/03', '12/1/04', '3/28/05', '4/2/05', '4/23/05', '10/8/05', '10/14/05', '10/25/05', '1/14/06', '4/12/07', '2/24/10', '2/25/10', '3/6/11', '10/14/11', '10/19/11', '10/29/11', '11/23/11', '12/7/11', '4/22/12', '10/19/12', '3/29/14', '3/30/14', '3/31/17', '12/25/20']


In [89]:
# Dates where it may have snowed
# '11/9/1996','11/1/1997','11/26/1998','11/2/1999', '11/27/1999', '1/10/2000','10/27/2003', '10/29/2003','11/20/2003','12/1/2004','1/14/2006','2/24/2010', '2/25/2010', '3/6/2011','10/29/2011', '11/23/2011', '12/7/2011',

In [90]:
#test = df.loc[((df['Date']=='11/8/1996'))]
#ninty_six = df[df['Date'].str.contains("1996")]
#nov_ninty_six = ninty_six[ninty_six['Date'].str.contains("11/")]
#nov_ninty_six[['Date','AvgTemperature', 'MinTemperature', 'MaxTemperature','Precipitation', 'Snowfall', 'SnowDepth']]

### Looking at March and April for Snowfall and SnowDepth

### March 1996
* From 3/15/1996 to 3/31/1996 and on Snowdepth is 0
* Precipitation was 0 from 3/27/1996 to 3/31/1996
* SnowFall was 0 from 3/9/1996 to 3/31/1996
### Because of that I will look at April 1996
* April 1996
* On 4/1/1996, 4/4/1996, 4/6/1996, 4/9/1996, 4/15/1996 and on 4/1/1996 and 4/9/1996 there was precipitation
* 4/1/1996 and 4/9/1996 are Missing Snowfall. MinTemperature was 32.0 and 26.0, respectively but Max and Avg are too warm.
* Input 0 for Snowfall and 0 for SnowDepth

In [91]:
x = df.loc[(df['Year'] == 1996) & (df['Month'] == 4)]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth
5935,4/1/96,62.0,32.0,47.0,0.31,0.0,0
5936,4/2/96,51.0,37.0,44.0,0.54,0.0,0
5937,4/3/96,54.0,34.0,44.0,0.00,0.0,0
5938,4/4/96,53.0,31.0,42.0,0.00,0.0,0
5939,4/5/96,50.0,36.0,43.0,0.00,0.0,0
5940,4/6/96,54.0,32.0,43.0,0.00,0.0,0
5941,4/7/96,45.0,34.0,39.5,0.26,0.0,0
5942,4/8/96,45.0,34.0,39.5,0.01,0.0,0
5943,4/9/96,47.0,26.0,36.5,0.15,0.0,0
5944,4/10/96,41.0,34.0,37.5,0.00,0.0,0


In [92]:
x = df.loc[(df['Year'] == 1996) & (df['Month'] == 4) & (df['MinTemperature'] < 33)]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth
5935,4/1/96,62.0,32.0,47.0,0.31,0.0,0
5938,4/4/96,53.0,31.0,42.0,0.00,0.0,0
5940,4/6/96,54.0,32.0,43.0,0.00,0.0,0
5943,4/9/96,47.0,26.0,36.5,0.15,0.0,0
5949,4/15/96,60.0,30.0,45.0,0.00,0.0,0


In [93]:
x = df.loc[(df['Year'] == 1996) & (df['Month'] == 4) & (df['Precipitation'] > 0.001)]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth
5935,4/1/96,62.0,32.0,47.0,0.31,0.0,0
5936,4/2/96,51.0,37.0,44.0,0.54,0.0,0
5941,4/7/96,45.0,34.0,39.5,0.26,0.0,0
5942,4/8/96,45.0,34.0,39.5,0.01,0.0,0
5943,4/9/96,47.0,26.0,36.5,0.15,0.0,0
5947,4/13/96,50.0,40.0,45.0,0.21,0.0,0
5948,4/14/96,46.0,37.0,41.5,0.14,0.0,0
5950,4/16/96,49.0,40.0,44.5,2.76,0.0,0
5957,4/23/96,84.0,56.0,70.0,0.08,0.0,0
5960,4/26/96,72.0,56.0,64.0,0.15,0.0,0


In [94]:
x = df.loc[(df['Year'] == 1996) & (df['Month'] == 4) & (df['Precipitation'] > 0.001) & (df['MinTemperature'] > 32)]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth
5936,4/2/96,51.0,37.0,44.0,0.54,0.0,0
5941,4/7/96,45.0,34.0,39.5,0.26,0.0,0
5942,4/8/96,45.0,34.0,39.5,0.01,0.0,0
5947,4/13/96,50.0,40.0,45.0,0.21,0.0,0
5948,4/14/96,46.0,37.0,41.5,0.14,0.0,0
5950,4/16/96,49.0,40.0,44.5,2.76,0.0,0
5957,4/23/96,84.0,56.0,70.0,0.08,0.0,0
5960,4/26/96,72.0,56.0,64.0,0.15,0.0,0
5961,4/27/96,58.0,37.0,47.5,0.01,0.0,0
5963,4/29/96,49.0,43.0,46.0,1.19,0.0,0


In [95]:
x = df.loc[(df['Year'] == 1996) & (df['Month'] == 12)]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth
6179,12/1/96,55.0,41.0,48.0,0.51,0.0,M
6180,12/2/96,58.0,30.0,44.0,1.57,0.0,M
6181,12/3/96,46.0,26.0,36.0,0.00,0.0,M
6182,12/4/96,39.0,31.0,35.0,0.03,0.0,M
6183,12/5/96,42.0,32.0,37.0,0.00,0.0,M
6184,12/6/96,39.0,33.0,36.0,0.41,0.0,M
6185,12/7/96,33.0,27.0,30.0,1.00,10.0,M
6186,12/8/96,41.0,23.0,32.0,0.12,1.2,M
6187,12/9/96,40.0,20.0,30.0,0.00,0.0,M
6188,12/10/96,40.0,18.0,29.0,0.00,0.0,M


In [96]:
x = df.loc[(df['Year'] == 1996) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth


### November 1996
* Three M on 11/25/1996, 11/26/1996 and 11/30/1996
* Precipitation for 11/25 and 11/30 were so low I have no problem putting Snowfall to 0
* MinTemperature for 11/26/2020 was 32 so I have no problem putting Snowfall to 0

In [97]:
x = df.loc[(df['Year'] == 1996) & (df['Month'] == 11)]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth
6149,11/1/96,47.0,32.0,39.5,0.00,0.0,M
6150,11/2/96,47.0,27.0,37.0,0.00,0.0,M
6151,11/3/96,47.0,30.0,38.5,0.00,0.0,M
6152,11/4/96,53.0,28.0,40.5,0.00,0.0,M
6153,11/5/96,57.0,39.0,48.0,0.00,0.0,M
6154,11/6/96,55.0,41.0,48.0,0.00,0.0,M
6155,11/7/96,62.0,46.0,54.0,0.06,0.0,M
6156,11/8/96,68.0,62.0,65.0,0.63,0.0,M
6157,11/9/96,67.0,39.0,53.0,1.70,0.0,M
6158,11/10/96,50.0,34.0,42.0,0.00,0.0,M


In [98]:
df.loc[(df['Year'] == 1996) & (df['Month'] == 11), 'SnowDepth'] = '0.0'

In [99]:
x = df.loc[(df['Year'] == 1996) & (df['Month'] == 11)]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth
6149,11/1/96,47.0,32.0,39.5,0.00,0.0,0.0
6150,11/2/96,47.0,27.0,37.0,0.00,0.0,0.0
6151,11/3/96,47.0,30.0,38.5,0.00,0.0,0.0
6152,11/4/96,53.0,28.0,40.5,0.00,0.0,0.0
6153,11/5/96,57.0,39.0,48.0,0.00,0.0,0.0
6154,11/6/96,55.0,41.0,48.0,0.00,0.0,0.0
6155,11/7/96,62.0,46.0,54.0,0.06,0.0,0.0
6156,11/8/96,68.0,62.0,65.0,0.63,0.0,0.0
6157,11/9/96,67.0,39.0,53.0,1.70,0.0,0.0
6158,11/10/96,50.0,34.0,42.0,0.00,0.0,0.0


### December 1996
* Missing Snowfall on:
    * 12/2/1996, 12/4/1996, 12/7/1996, 12/8/1996, 12/11/1996, 12/19/1996, 12/25/1996, 12/31/1996
* 12/2/1996: Too warm so did 0.0
* 12/4/1996: 0.03 precipitation so did 0.0
* 12/7/1996: Cold and Enough precipitation of 1.0 so will do 10
* 12/8/1996: Cold and Enough Precipitation of .12 so will do 1.2
* 12/11/1996: Too warm so 0.0
* 12/19/1996: Too warm so 0.0
* 12/25/1996: Not enough precipitation so did 0.0
* 12/31/1996: Not enough precipitation so did 0.0

In [100]:
x = df.loc[(df['Year'] == 1996) & (df['Month'] == 12)]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6179,12/1/96,55.0,41.0,48.0,0.51,0.0,M,1
6180,12/2/96,58.0,30.0,44.0,1.57,0.0,M,2
6181,12/3/96,46.0,26.0,36.0,0.00,0.0,M,3
6182,12/4/96,39.0,31.0,35.0,0.03,0.0,M,4
6183,12/5/96,42.0,32.0,37.0,0.00,0.0,M,5
6184,12/6/96,39.0,33.0,36.0,0.41,0.0,M,6
6185,12/7/96,33.0,27.0,30.0,1.00,10.0,M,7
6186,12/8/96,41.0,23.0,32.0,0.12,1.2,M,8
6187,12/9/96,40.0,20.0,30.0,0.00,0.0,M,9
6188,12/10/96,40.0,18.0,29.0,0.00,0.0,M,10


In [101]:
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 1), 'SnowDepth'] = '0.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 2), 'SnowDepth'] = '0.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 3), 'SnowDepth'] = '0.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 4), 'SnowDepth'] = '0.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 5), 'SnowDepth'] = '0.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 6), 'SnowDepth'] = '0.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 7), 'SnowDepth'] = '10.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 8), 'SnowDepth'] = '11.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 9), 'SnowDepth'] = '11.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 10), 'SnowDepth'] = '11.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 11), 'SnowDepth'] = '11.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 12), 'SnowDepth'] = '11.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 13), 'SnowDepth'] = '11.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 14), 'SnowDepth'] = '11.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 15), 'SnowDepth'] = '11.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 16), 'SnowDepth'] = '11.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 17), 'SnowDepth'] = '8.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 18), 'SnowDepth'] = '7.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 19), 'SnowDepth'] = '5.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 20), 'SnowDepth'] = '5.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 21), 'SnowDepth'] = '5.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 22), 'SnowDepth'] = '5.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 23), 'SnowDepth'] = '5.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 24), 'SnowDepth'] = '2.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 25), 'SnowDepth'] = '2.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 26), 'SnowDepth'] = '2.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 27), 'SnowDepth'] = '2.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 28), 'SnowDepth'] = '2.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 29), 'SnowDepth'] = '0.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 30), 'SnowDepth'] = '0.0'
df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Day'] == 31), 'SnowDepth'] = '0.0'

In [102]:
x = df.loc[(df['Year'] == 1996) & (df['Month'] == 12) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day


In [103]:
x = df.loc[(df['Year'] == 1996) & (df['Month'] == 12)]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6179,12/1/96,55.0,41.0,48.0,0.51,0.0,0.0,1
6180,12/2/96,58.0,30.0,44.0,1.57,0.0,0.0,2
6181,12/3/96,46.0,26.0,36.0,0.00,0.0,0.0,3
6182,12/4/96,39.0,31.0,35.0,0.03,0.0,0.0,4
6183,12/5/96,42.0,32.0,37.0,0.00,0.0,0.0,5
6184,12/6/96,39.0,33.0,36.0,0.41,0.0,0.0,6
6185,12/7/96,33.0,27.0,30.0,1.00,10.0,10.0,7
6186,12/8/96,41.0,23.0,32.0,0.12,1.2,11.0,8
6187,12/9/96,40.0,20.0,30.0,0.00,0.0,11.0,9
6188,12/10/96,40.0,18.0,29.0,0.00,0.0,11.0,10


### January 1997
* The below had missing values and based on the AvgTemperature and Precipitation I input a value for Snowfall
* 1/1/1997
* 1/2/1997
* 1/9/1997
* 1/10/1997
* 1/16/1997
* 1/22/1997
* 1/24/1997
* 1/25/1997
* 1/27/1997
* 1/28/1997
* 1/31/1997

In [104]:
x = df.loc[(df['Year'] == 1997) & (df['Month'] == 1) & (df['SnowDepth'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6210,1/1/97,19.0,-1.0,9.0,0.03,0.3,M,1
6211,1/2/97,36.0,19.0,27.5,0.09,0.9,M,2
6212,1/3/97,47.0,34.0,40.5,0.00,0.0,M,3
6213,1/4/97,47.0,33.0,40.0,0.00,0.0,M,4
6214,1/5/97,46.0,38.0,42.0,0.06,0.0,M,5
6215,1/6/97,41.0,31.0,36.0,0.00,0.0,M,6
6216,1/7/97,35.0,24.0,29.5,0.00,0.0,M,7
6217,1/8/97,31.0,23.0,27.0,0.00,0.0,M,8
6218,1/9/97,29.0,15.0,22.0,0.04,0.4,M,9
6219,1/10/97,39.0,28.0,33.5,0.19,0.0,M,10


In [105]:
df.loc[(df['Year'] == 1997) & (df['Month'] == 1) & (df['Day'] == 1) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.3'
df.loc[(df['Year'] == 1997) & (df['Month'] == 1) & (df['Day'] == 2) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.9'
df.loc[(df['Year'] == 1997) & (df['Month'] == 1) & (df['Day'] == 9) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.4'
df.loc[(df['Year'] == 1997) & (df['Month'] == 1) & (df['Day'] == 10) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 1) & (df['Day'] == 16) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 1) & (df['Day'] == 22) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 1) & (df['Day'] == 24) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.6'
df.loc[(df['Year'] == 1997) & (df['Month'] == 1) & (df['Day'] == 25) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 1) & (df['Day'] == 27) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.6'
df.loc[(df['Year'] == 1997) & (df['Month'] == 1) & (df['Day'] == 28) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 1) & (df['Day'] == 31) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.8'

In [106]:
s_t5 = df.loc[df['SnowDepth'] == 'T']
print(len(s_t5))
s_m5 = df.loc[df['SnowDepth'] == 'M']
print(len(s_m5))

0
2127


In [107]:
snowfalldeparture = df.loc[df['SnowfallDeparture'] == 'T']
print(len(snowfalldeparture))
snowfalldeparture = df.loc[df['SnowfallDeparture'] == 'M']
print(len(snowfalldeparture))

0
2222


# There is a lot of missing data for SnowDepth and SnowfallDeparture so I'm just going to export the dataframe as is then drop SnowDepth and SnowfallDeparture

In [108]:
df.to_csv('bradley_1_1_1980_10_14_2021_Clean_V2.csv')

### February 1997
* The following dates had missing values for Snowfall
* 2/1/1997, 2/3/1997, 2/4/1997, 2/5/1997, 2/11/1997, 2/14/1997, 2/15/1997, 2/16/1997, 2/22/1997, 2/26/1997

In [129]:
x = df.loc[(df['Year'] == 1997) & (df['Month'] == 2) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6241,2/1/1997,45.0,30.0,37.5,0.12,M,M,1
6243,2/3/1997,40.0,30.0,35.0,0.01,M,M,3
6244,2/4/1997,34.0,26.0,30.0,0.13,M,M,4
6245,2/5/1997,44.0,31.0,37.5,0.71,M,M,5
6251,2/11/1997,37.0,22.0,29.5,0.03,M,M,11
6254,2/14/1997,33.0,21.0,27.0,0.24,M,M,14
6255,2/15/1997,42.0,32.0,37.0,0.05,M,M,15
6256,2/16/1997,34.0,23.0,28.5,0.01,M,M,16
6262,2/22/1997,69.0,32.0,50.5,0.01,M,M,22
6266,2/26/1997,55.0,27.0,41.0,0.01,M,M,26


In [130]:
df.loc[(df['Year'] == 1997) & (df['Month'] == 2) & (df['Day'] == 1) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 2) & (df['Day'] == 3) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 2) & (df['Day'] == 4) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.3'
df.loc[(df['Year'] == 1997) & (df['Month'] == 2) & (df['Day'] == 5) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 2) & (df['Day'] == 11) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.3'
df.loc[(df['Year'] == 1997) & (df['Month'] == 2) & (df['Day'] == 14) & (df['Snowfall'] == 'M'), 'Snowfall'] = '2.4'
df.loc[(df['Year'] == 1997) & (df['Month'] == 2) & (df['Day'] == 15) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 2) & (df['Day'] == 16) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.1'
df.loc[(df['Year'] == 1997) & (df['Month'] == 2) & (df['Day'] == 22) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 2) & (df['Day'] == 26) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'

In [131]:
x = df.loc[(df['Year'] == 1997) & (df['Month'] == 2) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day


In [132]:
s_t6 = df.loc[df['Snowfall'] == 'T']
print(len(s_t6))
s_m6 = df.loc[df['Snowfall'] == 'M']
print(len(s_m6))

0
134


### March 1997
* The following dates had missing values for Snowfall
* 3/3/1997 3/4/1997 3/8/1997 3/10/1997 3/14/1997 3/15/1997 3/20/1997 3/22/1997 3/25/1997 3/31/1997

In [133]:
x = df.loc[(df['Year'] == 1997) & (df['Month'] == 3) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6271,3/3/1997,41.0,29.0,35.0,0.05,M,M,3
6272,3/4/1997,36.0,29.0,32.5,0.02,M,M,4
6276,3/8/1997,43.0,20.0,31.5,0.03,M,M,8
6278,3/10/1997,48.0,29.0,38.5,0.39,M,4,10
6282,3/14/1997,32.0,19.0,25.5,0.64,M,M,14
6283,3/15/1997,39.0,24.0,31.5,0.02,M,M,15
6288,3/20/1997,38.0,31.0,34.5,0.15,M,M,20
6290,3/22/1997,50.0,28.0,39.0,0.04,M,M,22
6293,3/25/1997,48.0,22.0,35.0,0.03,M,M,25
6299,3/31/1997,48.0,31.0,39.5,1.11,M,M,31


In [134]:
df.loc[(df['Year'] == 1997) & (df['Month'] == 3) & (df['Day'] == 3) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 3) & (df['Day'] == 4) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 3) & (df['Day'] == 8) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 3) & (df['Day'] == 10) & (df['Snowfall'] == 'M'), 'Snowfall'] = '4.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 3) & (df['Day'] == 14) & (df['Snowfall'] == 'M'), 'Snowfall'] = '6.4'
df.loc[(df['Year'] == 1997) & (df['Month'] == 3) & (df['Day'] == 15) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 3) & (df['Day'] == 20) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 3) & (df['Day'] == 22) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 3) & (df['Day'] == 25) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 3) & (df['Day'] == 31) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'

In [135]:
x = df.loc[(df['Year'] == 1997) & (df['Month'] == 3)]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6269,3/1/1997,44.0,33.0,38.5,0.00,0.0,M,1
6270,3/2/1997,59.0,38.0,48.5,0.15,0.0,M,2
6271,3/3/1997,41.0,29.0,35.0,0.05,0.0,M,3
6272,3/4/1997,36.0,29.0,32.5,0.02,0.0,M,4
6273,3/5/1997,48.0,33.0,40.5,0.28,0.0,M,5
6274,3/6/1997,44.0,33.0,38.5,0.13,0.0,M,6
6275,3/7/1997,37.0,23.0,30.0,0.00,0.0,M,7
6276,3/8/1997,43.0,20.0,31.5,0.03,0.0,M,8
6277,3/9/1997,34.0,18.0,26.0,0.00,0.0,M,9
6278,3/10/1997,48.0,29.0,38.5,0.39,4.0,4,10


In [136]:
s_t7 = df.loc[df['Snowfall'] == 'T']
print(len(s_t7))
s_m7 = df.loc[df['Snowfall'] == 'M']
print(len(s_m7))

0
124


### April 1997

In [137]:
x = df.loc[(df['Year'] == 1997) & (df['Month'] == 4) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6300,4/1/1997,43.0,31.0,37.0,0.06,M,M,1


In [138]:
df.loc[(df['Year'] == 1997) & (df['Month'] == 4) & (df['Day'] == 1) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'

In [139]:
x = df.loc[(df['Year'] == 1997) & (df['Month'] == 4) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day


### October 1997

In [140]:
x = df.loc[(df['Year'] == 1997)& (df['Month'] == 10) &(df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6508,10/26/1997,48.0,28.0,38.0,0.31,M,M,26
6513,10/31/1997,64.0,30.0,47.0,0.16,M,M,31


In [141]:
df.loc[(df['Year'] == 1997) & (df['Month'] == 10) & (df['Day'] == 26) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 10) & (df['Day'] == 31) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'

### November 1997

In [142]:
x = df.loc[(df['Year'] == 1997)& (df['Month'] == 11)& (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6527,11/14/1997,34.0,27.0,30.5,0.85,M,M,14
6534,11/21/1997,52.0,26.0,39.0,0.01,M,M,21
6536,11/23/1997,34.0,32.0,33.0,0.05,M,M,23
6540,11/27/1997,45.0,26.0,35.5,0.01,M,M,27
6541,11/28/1997,39.0,27.0,33.0,0.02,M,M,28
6543,11/30/1997,35.0,22.0,28.5,0.43,M,M,30


In [143]:
df.loc[(df['Year'] == 1997) & (df['Month'] == 11) & (df['Day'] == 14) & (df['Snowfall'] == 'M'), 'Snowfall'] = '8.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 11) & (df['Day'] == 21) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 11) & (df['Day'] == 23) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 11) & (df['Day'] == 27) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 11) & (df['Day'] == 28) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 11) & (df['Day'] == 30) & (df['Snowfall'] == 'M'), 'Snowfall'] = '4.0'

### December 1997

In [144]:
x = df.loc[(df['Year'] == 1997)& (df['Month'] == 12)& (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6544,12/1/1997,40.0,28.0,34.0,0.03,M,M,1
6547,12/4/1997,48.0,32.0,40.0,0.06,M,M,4
6553,12/10/1997,35.0,22.0,28.5,0.23,M,M,10
6555,12/12/1997,34.0,26.0,30.0,0.01,M,M,12
6566,12/23/1997,34.0,26.0,30.0,0.82,M,M,23
6568,12/25/1997,41.0,26.0,33.5,0.39,M,M,25
6570,12/27/1997,37.0,28.0,32.5,0.04,M,M,27
6572,12/29/1997,35.0,13.0,24.0,0.42,M,M,29
6573,12/30/1997,40.0,30.0,35.0,0.18,M,M,30


In [145]:
df.loc[(df['Year'] == 1997) & (df['Month'] == 12) & (df['Day'] == 1) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 12) & (df['Day'] == 4) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 12) & (df['Day'] == 10) & (df['Snowfall'] == 'M'), 'Snowfall'] = '2.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 12) & (df['Day'] == 12) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 12) & (df['Day'] == 23) & (df['Snowfall'] == 'M'), 'Snowfall'] = '8.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 12) & (df['Day'] == 25) & (df['Snowfall'] == 'M'), 'Snowfall'] = '3.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 12) & (df['Day'] == 27) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1997) & (df['Month'] == 12) & (df['Day'] == 29) & (df['Snowfall'] == 'M'), 'Snowfall'] = '4.2'
df.loc[(df['Year'] == 1997) & (df['Month'] == 12) & (df['Day'] == 30) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.9'

### January 1998

In [146]:
x = df.loc[(df['Year'] == 1998)& (df['Month'] == 1)& (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6587,1/13/1998,46.0,27.0,36.5,0.07,M,M,13
6589,1/15/1998,27.0,16.0,21.5,0.16,M,M,15
6590,1/16/1998,30.0,27.0,28.5,0.40,M,M,16
6593,1/19/1998,32.0,26.0,29.0,0.02,M,M,19
6597,1/23/1998,33.0,22.0,27.5,1.09,M,M,23
6598,1/24/1998,37.0,32.0,34.5,0.30,M,M,24


In [147]:
df.loc[(df['Year'] == 1998) & (df['Month'] == 1) & (df['Day'] == 13) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.7'
df.loc[(df['Year'] == 1998) & (df['Month'] == 1) & (df['Day'] == 15) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.6'
df.loc[(df['Year'] == 1998) & (df['Month'] == 1) & (df['Day'] == 16) & (df['Snowfall'] == 'M'), 'Snowfall'] = '4.0'
df.loc[(df['Year'] == 1998) & (df['Month'] == 1) & (df['Day'] == 19) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.2'
df.loc[(df['Year'] == 1998) & (df['Month'] == 1) & (df['Day'] == 23) & (df['Snowfall'] == 'M'), 'Snowfall'] = '10.9'
df.loc[(df['Year'] == 1998) & (df['Month'] == 1) & (df['Day'] == 24) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.3'

### February 1998

In [148]:
x = df.loc[(df['Year'] == 1998)& (df['Month'] == 2)& (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6609,2/4/1998,38.0,31.0,34.5,0.06,M,M,4
6610,2/5/1998,35.0,29.0,32.0,0.09,M,M,5
6622,2/17/1998,40.0,26.0,33.0,0.48,M,M,17


In [149]:
df.loc[(df['Year'] == 1998) & (df['Month'] == 2) & (df['Day'] == 4) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.6'
df.loc[(df['Year'] == 1998) & (df['Month'] == 2) & (df['Day'] == 5) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.9'
df.loc[(df['Year'] == 1998) & (df['Month'] == 2) & (df['Day'] == 17) & (df['Snowfall'] == 'M'), 'Snowfall'] = '4.8'

### March 1998

In [150]:
x = df.loc[(df['Year'] == 1998)& (df['Month'] == 3) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6638,3/5/1998,45.0,28.0,36.5,0.02,M,M,5
6641,3/8/1998,48.0,31.0,39.5,0.50,M,M,8
6647,3/14/1998,37.0,26.0,31.5,0.13,M,M,14
6651,3/18/1998,37.0,30.0,33.5,0.14,M,M,18
6654,3/21/1998,37.0,28.0,32.5,0.26,M,M,21
6655,3/22/1998,33.0,27.0,30.0,0.07,M,M,22


In [151]:
df.loc[(df['Year'] == 1998) & (df['Month'] == 3) & (df['Day'] == 5) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.2'
df.loc[(df['Year'] == 1998) & (df['Month'] == 3) & (df['Day'] == 8) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1998) & (df['Month'] == 3) & (df['Day'] == 14) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.3'
df.loc[(df['Year'] == 1998) & (df['Month'] == 3) & (df['Day'] == 18) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.4'
df.loc[(df['Year'] == 1998) & (df['Month'] == 3) & (df['Day'] == 21) & (df['Snowfall'] == 'M'), 'Snowfall'] = '2.6'
df.loc[(df['Year'] == 1998) & (df['Month'] == 3) & (df['Day'] == 22) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.7'

### November and December 1998

In [152]:
x = df.loc[(df['Year'] == 1998)& (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6888,11/10/1998,46.0,22.0,34.0,0.04,M,M,10
6897,11/19/1998,41.0,24.0,32.5,0.01,M,M,19
6925,12/17/1998,40.0,31.0,35.5,0.06,M,M,17
6930,12/22/1998,58.0,20.0,39.0,0.25,M,M,22
6937,12/29/1998,35.0,27.0,31.0,0.30,M,M,29
6938,12/30/1998,34.0,10.0,22.0,0.04,M,M,30


In [153]:
df.loc[(df['Year'] == 1998) & (df['Month'] == 11) & (df['Day'] == 10) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.4'
df.loc[(df['Year'] == 1998) & (df['Month'] == 11) & (df['Day'] == 19) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1998) & (df['Month'] == 12) & (df['Day'] == 17) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1998) & (df['Month'] == 12) & (df['Day'] == 22) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1998) & (df['Month'] == 12) & (df['Day'] == 29) & (df['Snowfall'] == 'M'), 'Snowfall'] = '3.0'
df.loc[(df['Year'] == 1998) & (df['Month'] == 12) & (df['Day'] == 30) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.4'

In [154]:
s_t8 = df.loc[df['Snowfall'] == 'T']
print(len(s_t8))
s_m8 = df.loc[df['Snowfall'] == 'M']
print(len(s_m8))

0
85


### January 1999

In [155]:
x = df.loc[(df['Year'] == 1999) & (df['Month'] == 1) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6942,1/3/1999,34.0,16.0,25.0,1.34,M,M,3
6947,1/8/1999,22.0,5.0,13.5,0.31,M,M,8
6948,1/9/1999,47.0,21.0,34.0,0.41,M,M,9
6951,1/12/1999,41.0,17.0,29.0,0.06,M,M,12
6952,1/13/1999,43.0,10.0,26.5,0.01,M,M,13
6953,1/14/1999,9.0,2.0,5.5,0.11,M,M,14
6954,1/15/1999,35.0,9.0,22.0,1.13,M,M,15
6957,1/18/1999,53.0,25.0,39.0,0.58,M,M,18
6960,1/21/1999,37.0,21.0,29.0,0.22,M,M,21
6967,1/28/1999,33.0,23.0,28.0,0.10,M,M,28


In [156]:
df.loc[(df['Year'] == 1999) & (df['Month'] == 1) & (df['Day'] == 3) & (df['Snowfall'] == 'M'), 'Snowfall'] = '13.4'
df.loc[(df['Year'] == 1999) & (df['Month'] == 1) & (df['Day'] == 8) & (df['Snowfall'] == 'M'), 'Snowfall'] = '3.1'
df.loc[(df['Year'] == 1999) & (df['Month'] == 1) & (df['Day'] == 9) & (df['Snowfall'] == 'M'), 'Snowfall'] = '4.1'
df.loc[(df['Year'] == 1999) & (df['Month'] == 1) & (df['Day'] == 12) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.6'
df.loc[(df['Year'] == 1999) & (df['Month'] == 1) & (df['Day'] == 13) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.1'
df.loc[(df['Year'] == 1999) & (df['Month'] == 1) & (df['Day'] == 14) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.1'
df.loc[(df['Year'] == 1999) & (df['Month'] == 1) & (df['Day'] == 15) & (df['Snowfall'] == 'M'), 'Snowfall'] = '11.3'
df.loc[(df['Year'] == 1999) & (df['Month'] == 1) & (df['Day'] == 18) & (df['Snowfall'] == 'M'), 'Snowfall'] = '5.8'
df.loc[(df['Year'] == 1999) & (df['Month'] == 1) & (df['Day'] == 21) & (df['Snowfall'] == 'M'), 'Snowfall'] = '2.2'
df.loc[(df['Year'] == 1999) & (df['Month'] == 1) & (df['Day'] == 28) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 1) & (df['Day'] == 29) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.1'

### Febuary 1999

In [157]:
x = df.loc[(df['Year'] == 1999) & (df['Month'] == 2) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
6972,2/2/1999,46.0,30.0,38.0,1.39,M,M,2
6974,2/4/1999,40.0,25.0,32.5,0.18,M,M,4
6977,2/7/1999,39.0,30.0,34.5,0.05,M,M,7
6988,2/18/1999,38.0,32.0,35.0,0.78,M,M,18
6995,2/25/1999,29.0,23.0,26.0,0.02,M,M,25
6996,2/26/1999,43.0,27.0,35.0,0.05,M,M,26
6998,2/28/1999,38.0,21.0,29.5,0.78,M,M,28


In [158]:
df.loc[(df['Year'] == 1999) & (df['Month'] == 2) & (df['Day'] == 2) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 2) & (df['Day'] == 4) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.8'
df.loc[(df['Year'] == 1999) & (df['Month'] == 2) & (df['Day'] == 7) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 2) & (df['Day'] == 18) & (df['Snowfall'] == 'M'), 'Snowfall'] = '4.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 2) & (df['Day'] == 25) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.2'
df.loc[(df['Year'] == 1999) & (df['Month'] == 2) & (df['Day'] == 26) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 2) & (df['Day'] == 28) & (df['Snowfall'] == 'M'), 'Snowfall'] = '7.8'

### March 1999

In [159]:
x = df.loc[(df['Year'] == 1999) & (df['Month'] == 3) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
7001,3/3/1999,55.0,23.0,39.0,0.48,M,M,3
7002,3/4/1999,56.0,30.0,43.0,0.71,M,M,4
7004,3/6/1999,36.0,24.0,30.0,0.26,M,M,6
7005,3/7/1999,27.0,15.0,21.0,0.01,M,M,7
7012,3/14/1999,46.0,29.0,37.5,0.22,M,M,14
7013,3/15/1999,37.0,31.0,34.0,0.20,M,M,15
7019,3/21/1999,51.0,22.0,36.5,0.30,M,M,21
7022,3/24/1999,50.0,27.0,38.5,0.06,M,M,24
7025,3/27/1999,58.0,27.0,42.5,0.02,M,M,27


In [160]:
df.loc[(df['Year'] == 1999) & (df['Month'] == 3) & (df['Day'] == 3) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 3) & (df['Day'] == 4) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 3) & (df['Day'] == 6) & (df['Snowfall'] == 'M'), 'Snowfall'] = '2.6'
df.loc[(df['Year'] == 1999) & (df['Month'] == 3) & (df['Day'] == 7) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.1'
df.loc[(df['Year'] == 1999) & (df['Month'] == 3) & (df['Day'] == 14) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 3) & (df['Day'] == 15) & (df['Snowfall'] == 'M'), 'Snowfall'] = '2.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 3) & (df['Day'] == 21) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 3) & (df['Day'] == 24) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 3) & (df['Day'] == 27) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'

### April 1999

In [161]:
x = df.loc[(df['Year'] == 1999) & (df['Month'] == 4) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
7040,4/11/1999,53.0,28.0,40.5,0.14,M,M,11


In [162]:
df.loc[(df['Year'] == 1999) & (df['Month'] == 4) & (df['Day'] == 11) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'

### October, November and December 1999

In [163]:
x = df.loc[(df['Year'] == 1999) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
7234,10/22/1999,63.0,31.0,47.0,0.31,M,M,22
7255,11/12/1999,46.0,25.0,35.5,0.01,M,M,12
7283,12/10/1999,53.0,24.0,38.5,0.15,M,M,10
7286,12/13/1999,42.0,23.0,32.5,0.09,M,M,13
7293,12/20/1999,38.0,19.0,28.5,0.86,M,M,20


In [164]:
df.loc[(df['Year'] == 1999) & (df['Month'] == 10) & (df['Day'] == 22) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 11) & (df['Day'] == 12) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 12) & (df['Day'] == 10) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 1999) & (df['Month'] == 12) & (df['Day'] == 13) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.9'
df.loc[(df['Year'] == 1999) & (df['Month'] == 12) & (df['Day'] == 20) & (df['Snowfall'] == 'M'), 'Snowfall'] = '8.6'

In [165]:
s_t9 = df.loc[df['Snowfall'] == 'T']
print(len(s_t9))
s_m9 = df.loc[df['Snowfall'] == 'M']
print(len(s_m9))

0
52


In [166]:
x = df.loc[(df['Year'] == 2000) & (df['Month'] == 1) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
7306,1/2/2000,54.0,30.0,42.0,0.01,M,M,2
7309,1/5/2000,40.0,20.0,30.0,0.02,M,M,5
7313,1/9/2000,52.0,29.0,40.5,0.09,M,M,9
7317,1/13/2000,31.0,11.0,21.0,0.21,M,M,13
7324,1/20/2000,19.0,2.0,10.5,0.01,M,M,20
7329,1/25/2000,27.0,20.0,23.5,0.20,M,M,25
7330,1/26/2000,29.0,15.0,22.0,0.01,M,M,26
7334,1/30/2000,32.0,1.0,16.5,0.03,M,M,30
7335,1/31/2000,37.0,26.0,31.5,0.37,M,M,31


In [167]:
df.loc[(df['Year'] == 2000) & (df['Month'] == 1) & (df['Day'] == 2) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2000) & (df['Month'] == 1) & (df['Day'] == 5) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.2'
df.loc[(df['Year'] == 2000) & (df['Month'] == 1) & (df['Day'] == 9) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2000) & (df['Month'] == 1) & (df['Day'] == 13) & (df['Snowfall'] == 'M'), 'Snowfall'] = '2.1'
df.loc[(df['Year'] == 2000) & (df['Month'] == 1) & (df['Day'] == 20) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.1'
df.loc[(df['Year'] == 2000) & (df['Month'] == 1) & (df['Day'] == 25) & (df['Snowfall'] == 'M'), 'Snowfall'] = '2.0'
df.loc[(df['Year'] == 2000) & (df['Month'] == 1) & (df['Day'] == 26) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.1'
df.loc[(df['Year'] == 2000) & (df['Month'] == 1) & (df['Day'] == 30) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.3'
df.loc[(df['Year'] == 2000) & (df['Month'] == 1) & (df['Day'] == 31) & (df['Snowfall'] == 'M'), 'Snowfall'] = '3.7'

In [168]:
x = df.loc[(df['Year'] == 2000) & (df['Month'] == 2) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
7336,2/1/2000,35.0,24.0,29.5,0.02,M,M,1
7338,2/3/2000,25.0,13.0,19.0,0.05,M,M,3
7339,2/4/2000,29.0,21.0,25.0,0.01,M,M,4
7346,2/11/2000,39.0,25.0,32.0,0.02,M,M,11
7348,2/13/2000,36.0,8.0,22.0,0.06,M,M,13
7349,2/14/2000,46.0,32.0,39.0,1.02,M,M,14
7353,2/18/2000,26.0,15.0,20.5,0.29,M,M,18
7354,2/19/2000,32.0,24.0,28.0,0.11,M,M,19
7355,2/20/2000,37.0,28.0,32.5,0.01,M,M,20


In [169]:
df.loc[(df['Year'] == 2000) & (df['Month'] == 2) & (df['Day'] == 1) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.2'
df.loc[(df['Year'] == 2000) & (df['Month'] == 2) & (df['Day'] == 3) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.5'
df.loc[(df['Year'] == 2000) & (df['Month'] == 2) & (df['Day'] == 4) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.1'
df.loc[(df['Year'] == 2000) & (df['Month'] == 2) & (df['Day'] == 11) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.2'
df.loc[(df['Year'] == 2000) & (df['Month'] == 2) & (df['Day'] == 13) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.6'
df.loc[(df['Year'] == 2000) & (df['Month'] == 2) & (df['Day'] == 14) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2000) & (df['Month'] == 2) & (df['Day'] == 18) & (df['Snowfall'] == 'M'), 'Snowfall'] = '2.9'
df.loc[(df['Year'] == 2000) & (df['Month'] == 2) & (df['Day'] == 19) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.1'
df.loc[(df['Year'] == 2000) & (df['Month'] == 2) & (df['Day'] == 20) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'

In [170]:
x = df.loc[(df['Year'] == 2000) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
7401,4/6/2000,66.0,32.0,49.0,0.01,M,M,6
7404,4/9/2000,57.0,31.0,44.0,0.80,M,M,9
7406,4/11/2000,42.0,29.0,35.5,0.01,M,M,11
7635,11/26/2000,42.0,25.0,33.5,0.63,M,M,26
7638,11/29/2000,49.0,32.0,40.5,0.02,M,M,29
7639,11/30/2000,43.0,32.0,37.5,0.24,M,M,30


In [171]:
df.loc[(df['Year'] == 2000) & (df['Month'] == 4) & (df['Day'] == 6) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2000) & (df['Month'] == 4) & (df['Day'] == 9) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2000) & (df['Month'] == 4) & (df['Day'] == 11) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.1'
df.loc[(df['Year'] == 2000) & (df['Month'] == 11) & (df['Day'] == 26) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2000) & (df['Month'] == 11) & (df['Day'] == 29) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2000) & (df['Month'] == 11) & (df['Day'] == 30) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'

In [172]:
s_t10 = df.loc[df['Snowfall'] == 'T']
print(len(s_t10))
s_m10 = df.loc[df['Snowfall'] == 'M']
print(len(s_m10))

0
28


In [173]:
x = df.loc[(df['Year'] == 2001) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
7974,10/31/2001,42.0,22.0,32.0,0.05,M,M,31
7983,11/9/2001,53.0,32.0,42.5,0.04,M,M,9
8001,11/27/2001,54.0,32.0,43.0,0.01,M,M,27


In [174]:
df.loc[(df['Year'] == 2001) & (df['Month'] == 10) & (df['Day'] == 31) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.5'
df.loc[(df['Year'] == 2001) & (df['Month'] == 11) & (df['Day'] == 9) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2001) & (df['Month'] == 11) & (df['Day'] == 27) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'

### January 2002

In [175]:
x = df.loc[(df['Year'] == 2002) & (df['Month'] == 1) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
8044,1/9/2002,37.0,24.0,30.5,0.03,M,M,9
8048,1/13/2002,40.0,31.0,35.5,0.18,M,M,13
8052,1/17/2002,43.0,28.0,35.5,0.01,M,M,17
8066,1/31/2002,36.0,29.0,32.5,0.13,M,M,31


In [176]:
df.loc[(df['Year'] == 2002) & (df['Month'] == 1) & (df['Day'] == 9) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.3'
df.loc[(df['Year'] == 2002) & (df['Month'] == 1) & (df['Day'] == 13) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2002) & (df['Month'] == 1) & (df['Day'] == 17) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2002) & (df['Month'] == 1) & (df['Day'] == 31) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.3'

### February 2002

In [177]:
x = df.loc[(df['Year'] == 2002) & (df['Month'] == 2) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
8067,2/1/2002,39.0,30.0,34.5,0.18,M,M,1
8076,2/10/2002,50.0,18.0,34.0,0.39,M,M,10
8077,2/11/2002,53.0,15.0,34.0,0.09,M,M,11
8083,2/17/2002,39.0,30.0,34.5,0.15,M,M,17
8086,2/20/2002,54.0,32.0,43.0,0.25,M,M,20
8093,2/27/2002,54.0,26.0,40.0,0.16,M,M,27


In [178]:
df.loc[(df['Year'] == 2002) & (df['Month'] == 2) & (df['Day'] == 1) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.8'
df.loc[(df['Year'] == 2002) & (df['Month'] == 2) & (df['Day'] == 10) & (df['Snowfall'] == 'M'), 'Snowfall'] = '3.9'
df.loc[(df['Year'] == 2002) & (df['Month'] == 2) & (df['Day'] == 11) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.9'
df.loc[(df['Year'] == 2002) & (df['Month'] == 2) & (df['Day'] == 17) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.5'
df.loc[(df['Year'] == 2002) & (df['Month'] == 2) & (df['Day'] == 20) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2002) & (df['Month'] == 2) & (df['Day'] == 27) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'

In [179]:
x = df.loc[(df['Year'] == 2002) & (df['Month'] == 3) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
8096,3/2/2002,47.0,19.0,33.0,0.07,M,M,2
8104,3/10/2002,60.0,26.0,43.0,0.36,M,M,10
8110,3/16/2002,53.0,28.0,40.5,0.28,M,M,16
8112,3/18/2002,37.0,30.0,33.5,0.17,M,M,18
8115,3/21/2002,51.0,27.0,39.0,0.01,M,M,21
8120,3/26/2002,38.0,32.0,35.0,0.51,M,M,26


In [180]:
df.loc[(df['Year'] == 2002) & (df['Month'] == 3) & (df['Day'] == 2) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.7'
df.loc[(df['Year'] == 2002) & (df['Month'] == 3) & (df['Day'] == 10) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2002) & (df['Month'] == 3) & (df['Day'] == 16) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2002) & (df['Month'] == 3) & (df['Day'] == 18) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.7'
df.loc[(df['Year'] == 2002) & (df['Month'] == 3) & (df['Day'] == 21) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2002) & (df['Month'] == 3) & (df['Day'] == 26) & (df['Snowfall'] == 'M'), 'Snowfall'] = '5.1'

In [181]:
x = df.loc[(df['Year'] == 2002) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
8137,4/12/2002,65.0,30.0,47.5,0.01,M,M,12
8331,10/23/2002,49.0,32.0,40.5,0.05,M,M,23
8333,10/25/2002,52.0,28.0,40.0,0.02,M,M,25
8343,11/4/2002,49.0,29.0,39.0,0.07,M,M,4
8344,11/5/2002,48.0,30.0,39.0,0.05,M,M,5
8355,11/16/2002,49.0,32.0,40.5,0.47,M,M,16
8356,11/17/2002,37.0,31.0,34.0,1.39,M,M,17
8360,11/21/2002,41.0,28.0,34.5,0.14,M,M,21
8362,11/23/2002,44.0,31.0,37.5,0.01,M,M,23


In [182]:
df.loc[(df['Year'] == 2002) & (df['Month'] == 4) & (df['Day'] == 12) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2002) & (df['Month'] == 10) & (df['Day'] == 23) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2002) & (df['Month'] == 10) & (df['Day'] == 25) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2002) & (df['Month'] == 11) & (df['Day'] == 4) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2002) & (df['Month'] == 11) & (df['Day'] == 5) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2002) & (df['Month'] == 11) & (df['Day'] == 16) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2002) & (df['Month'] == 11) & (df['Day'] == 17) & (df['Snowfall'] == 'M'), 'Snowfall'] = '13.9'
df.loc[(df['Year'] == 2002) & (df['Month'] == 11) & (df['Day'] == 21) & (df['Snowfall'] == 'M'), 'Snowfall'] = '1.4'
df.loc[(df['Year'] == 2002) & (df['Month'] == 11) & (df['Day'] == 23) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'

In [183]:
s_t11 = df.loc[df['Snowfall'] == 'T']
print(len(s_t11))
s_m11 = df.loc[df['Snowfall'] == 'M']
print(len(s_m11))

0
0


# So now there are no missing values for Snowfall for 1.1.1980 to 12.1.2020

## I had combined 12.2.2020 to 12.27.2020 with 1.1.1980 to 12.1.2020 so check if anything is off

### for 12.2.2020 to 12.27.2020 I know these were AtObsTemperature is all M, Precipitation has a few T's, Snowfall has a few T's and SnowDepth has a few M

In [184]:
x = df.loc[(df['Year'] == 2020) & (df['Month'] == 12)]
x

,Date,MaxTemperature,MaxTemperatureNormal,MaxTemperatureDeparture,MinTemperature,MinTemperatureNormal,MinTemperatureDeparture,AvgTemperature,AvgTemperatureNormal,AvgTemperatureDeparture,...,HDDNormal,HDDDeparture,CDD,CDDNormal,CDDDeparture,GDD,Precipitation,Year,Month,Day
14945,12/1/2020,62.0,45.2,16.8,38.0,28.7,9.3,50.0,36.9,13.1,...,28.0,-13.0,0.0,0.0,0.0,0.0,0.00,2020,12,1
0,12/2/2020,44.0,44.8,-0.8,33.0,28.3,4.7,38.5,36.5,2.0,...,28.0,-2.0,0.0,0.0,0.0,0.0,0.00,2020,12,2
1,12/3/2020,50.0,44.4,5.6,28.0,28.0,0.0,39.0,36.2,2.8,...,29.0,-3.0,0.0,0.0,0.0,0.0,0.00,2020,12,3
2,12/4/2020,57.0,43.9,13.1,40.0,27.6,12.4,48.5,35.8,12.7,...,29.0,-13.0,0.0,0.0,0.0,0.0,0.00,2020,12,4
3,12/5/2020,44.0,43.5,0.5,33.0,27.2,5.8,38.5,35.4,3.1,...,30.0,-4.0,0.0,0.0,0.0,0.0,1.67,2020,12,5
4,12/6/2020,37.0,43.1,-6.1,28.0,26.9,1.1,32.5,35.0,-2.5,...,30.0,2.0,0.0,0.0,0.0,0.0,0.00,2020,12,6
5,12/7/2020,39.0,42.7,-3.7,26.0,26.5,-0.5,32.5,34.6,-2.1,...,30.0,2.0,0.0,0.0,0.0,0.0,0.00,2020,12,7
6,12/8/2020,32.0,42.3,-10.3,25.0,26.1,-1.1,28.5,34.2,-5.7,...,31.0,5.0,0.0,0.0,0.0,0.0,0.00,2020,12,8
7,12/9/2020,38.0,41.9,-3.9,20.0,25.8,-5.8,29.0,33.8,-4.8,...,31.0,5.0,0.0,0.0,0.0,0.0,0.00,2020,12,9
8,12/10/2020,47.0,41.5,5.5,28.0,25.4,2.6,37.5,33.5,4.0,...,32.0,-5.0,0.0,0.0,0.0,0.0,0.00,2020,12,10


### AtObsTemperature for 12.2.2020 to 12.27.2020 all 'M'
* That's fine because most of it is M

In [187]:
print(df2['AtObsTemperature'].nunique())
obs = df2['AtObsTemperature'].unique()
obs_sort = np.sort(obs)
print(obs_sort)

1
['M']


In [186]:
x = df.loc[(df['Year'] == 2020) & (df['Month'] == 12) & (df['AtObsTemperature'] == 'M')]
x

,Date,MaxTemperature,MaxTemperatureNormal,MaxTemperatureDeparture,MinTemperature,MinTemperatureNormal,MinTemperatureDeparture,AvgTemperature,AvgTemperatureNormal,AvgTemperatureDeparture,...,HDDNormal,HDDDeparture,CDD,CDDNormal,CDDDeparture,GDD,Precipitation,Year,Month,Day
14945,12/1/2020,62.0,45.2,16.8,38.0,28.7,9.3,50.0,36.9,13.1,...,28.0,-13.0,0.0,0.0,0.0,0.0,0.00,2020,12,1
0,12/2/2020,44.0,44.8,-0.8,33.0,28.3,4.7,38.5,36.5,2.0,...,28.0,-2.0,0.0,0.0,0.0,0.0,0.00,2020,12,2
1,12/3/2020,50.0,44.4,5.6,28.0,28.0,0.0,39.0,36.2,2.8,...,29.0,-3.0,0.0,0.0,0.0,0.0,0.00,2020,12,3
2,12/4/2020,57.0,43.9,13.1,40.0,27.6,12.4,48.5,35.8,12.7,...,29.0,-13.0,0.0,0.0,0.0,0.0,0.00,2020,12,4
3,12/5/2020,44.0,43.5,0.5,33.0,27.2,5.8,38.5,35.4,3.1,...,30.0,-4.0,0.0,0.0,0.0,0.0,1.67,2020,12,5
4,12/6/2020,37.0,43.1,-6.1,28.0,26.9,1.1,32.5,35.0,-2.5,...,30.0,2.0,0.0,0.0,0.0,0.0,0.00,2020,12,6
5,12/7/2020,39.0,42.7,-3.7,26.0,26.5,-0.5,32.5,34.6,-2.1,...,30.0,2.0,0.0,0.0,0.0,0.0,0.00,2020,12,7
6,12/8/2020,32.0,42.3,-10.3,25.0,26.1,-1.1,28.5,34.2,-5.7,...,31.0,5.0,0.0,0.0,0.0,0.0,0.00,2020,12,8
7,12/9/2020,38.0,41.9,-3.9,20.0,25.8,-5.8,29.0,33.8,-4.8,...,31.0,5.0,0.0,0.0,0.0,0.0,0.00,2020,12,9
8,12/10/2020,47.0,41.5,5.5,28.0,25.4,2.6,37.5,33.5,4.0,...,32.0,-5.0,0.0,0.0,0.0,0.0,0.00,2020,12,10


In [189]:
x = df.loc[(df['Year'] == 2020) & (df['Month'] == 12) & (df['Precipitation'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day


In [190]:
x = df.loc[(df['Year'] == 2020) & (df['Month'] == 12) & (df['Precipitation'] == 'T')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day


In [191]:
x = df.loc[(df['Year'] == 2020) & (df['Month'] == 12) & (df['Snowfall'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day


In [192]:
x = df.loc[(df['Year'] == 2020) & (df['Month'] == 12) & (df['Snowfall'] == 'T')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day


In [193]:
x = df.loc[(df['Year'] == 2020) & (df['Month'] == 12) & (df['SnowDepth'] == 'T')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day


In [194]:
x = df.loc[(df['Year'] == 2020) & (df['Month'] == 12) & (df['SnowDepth'] == 'M')]
x[['Date', 'MaxTemperature', 'MinTemperature', 'AvgTemperature', 'Precipitation', 'Snowfall', 'SnowDepth', 'Day']]

,Date,MaxTemperature,MinTemperature,AvgTemperature,Precipitation,Snowfall,SnowDepth,Day
23,12/25/2020,63.0,35.0,49.0,2.12,0,M,25
25,12/27/2020,39.0,20.0,29.5,0.00,0,M,27


In [195]:
df.loc[(df['Year'] == 2020) & (df['Month'] == 3) & (df['Day'] == 25) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'
df.loc[(df['Year'] == 2020) & (df['Month'] == 3) & (df['Day'] == 27) & (df['Snowfall'] == 'M'), 'Snowfall'] = '0.0'

## Now df is all clean, and 12.2.2020 to 12.27.2020 is clean for what I am looking at

In [197]:
df[['Snowfall']] = df[['Snowfall']].astype(float)

### Convert Date from object datetime

In [199]:
#import datetime as dt
#df = pd.DataFrame(df['Date'])
#df1['ds'] = pd.to_datetime(df['Date'])
#df1['ds'] = df['Date'].map(dt.datetime.fromordinal)
#df1
df[['Date']] = df[['Date']].apply(pd.to_datetime)

### Any nans
* No nans

In [201]:
print(df.isna().sum())

Date                       0
MaxTemperature             0
MaxTemperatureNormal       0
MaxTemperatureDeparture    0
MinTemperature             0
MinTemperatureNormal       0
MinTemperatureDeparture    0
AvgTemperature             0
AvgTemperatureNormal       0
AvgTemperatureDeparture    0
AtObsTemperature           0
PrecipitationNormal        0
PrecipitationDeparture     0
Snowfall                   0
SnowfallNormal             0
SnowfallDeparture          0
SnowDepth                  0
HDD                        0
HDDNormal                  0
HDDDeparture               0
CDD                        0
CDDNormal                  0
CDDDeparture               0
GDD                        0
Precipitation              0
Year                       0
Month                      0
Day                        0
dtype: int64


In [202]:
df.to_csv('Bradley_1.1.1980_12.27.2020_Clean.csv')